# A Guide to Production-Grade RAG: From Theory to Autonomous Agents

## Table of Contents

**Part 1: Setting the Stage - Foundations and Our Core Challenge**
* [1.1. Introduction: The Limits of "Shallow" RAG](#part1-1-intro-pro)
* [1.2. Environment Setup: API Keys, Imports, and Configuration](#part1-2-env-pro-adv)
* [1.3. The Dataset: Preparing Our Knowledge Base](#part1-3-data-pro)
* [1.4. The Upgraded Challenge: A Multi-Source, Multi-Hop Query](#part1-4-challenge-pro-adv)

**Part 2: The Baseline - Building and Breaking a "Vanilla" RAG Pipeline**
* [2.1. Code Dependency: Document Loading and Naive Chunking](#part2-1-dep-pro)
* [2.2. Code Dependency: Creating the Vector Store](#part2-2-dep-pro)
* [2.3. Code Dependency: Assembling the Simple RAG Chain](#part2-3-dep-pro)
* [2.4. The Critical Failure Case: Demonstrating the Need for Advanced Techniques](#part2-4-fail-pro-adv)
* [2.5. Diagnosis: Why Did It Fail?](#part2-5-diag-pro-adv)

**Part 3: The "Deep Thinking" Upgrade: Engineering an Autonomous Reasoning Engine**
* [3.1. Code Dependency: Defining the `RAGState`](#part3-1-state-pro-adv)
* [3.2. Component 1: Dynamic Planning and Query Formulation](#part3-2-planner-pro-adv)
    * [3.2.1. The Tool-Aware Planner Agent](#part3-2-1-planner-pro-adv)
    * [3.2.2. Query Rewriting and Expansion](#part3-2-2-rewriter-pro)
    * [3.2.3. Entity and Constraint Extraction](#part3-2-3-metadata-pro)
* [3.3. Component 2: The Multi-Stage, Adaptive Retrieval Funnel](#part3-3-retrieval-pro-adv)
    * [3.3.1. NEW: The Retrieval Supervisor Agent](#part3-3-1-supervisor-pro)
    * [3.3.2. Implementing the Retrieval Strategies](#part3-3-2-strategies-pro)
    * [3.3.3. Stage 2 (High Precision): Cross-Encoder Reranker](#part3-3-3-reranker-pro)
    * [3.3.4. Stage 3 (Contextual Distillation)](#part3-3-4-distill-pro)
* [3.4. Component 3: Tool Augmentation with Web Search](#part3-4-tool-pro)
* [3.5. Component 4: The Self-Critique and Control Flow Policy](#part3-5-critique-pro)
    * [3.5.1. The "Update and Reflect" Step](#part3-5-1-reflect-pro)
    * [3.5.2. Policy Implementation (LLM-as-a-Judge)](#part3-5-2-policy-pro)
    * [3.5.3. Defining Robust Stopping Criteria](#part3-5-3-stopping-pro)

**Part 4: Assembly with LangGraph - Orchestrating the Reasoning Loop**
* [4.1. Code Dependency: Defining the Graph Nodes](#part4-1-nodes-pro-adv)
* [4.2. Code Dependency: Defining the Conditional Edges](#part4-2-edges-pro-adv)
* [4.3. Building the `StateGraph`](#part4-3-build-pro-adv)
* [4.4. Compiling and Visualizing the Workflow](#part4-4-viz-pro-adv)

**Part 5: Redemption - Running the Advanced Agent**
* [5.1. Invoking the Graph: A Step-by-Step Trace](#part5-1-invoke-pro-adv)
* [5.2. Analyzing the Final High-Quality Output](#part5-2-analyze-pro-adv)
* [5.3. Side-by-Side Comparison: Vanilla vs. Deep Thinking RAG](#part5-3-compare-pro-adv)

**Part 6: A Production-Grade Evaluation Framework**
* [6.1. Evaluation Metrics Overview](#part6-metrics-pro)
* [6.2. Code Dependency: Implementing Evaluation with RAGAs](#part6-4-ragas-code-pro-adv)
* [6.3. Interpreting the Evaluation Scores](#part6-5-interpret-pro-adv)

**Part 7: Optimizations and Production Considerations**
* [7.1. Optimization: Caching](#part7-1-cache-pro)
* [7.2. Feature: Provenance and Citations](#part7-2-provenance-pro)
* [7.3. Discussion: The Next Level - MDPs and Learned Policies](#part7-3-discussion-pro)
* [7.4. Handling Failure: Graceful Exits and Fallbacks](#part7-4-failure-pro)

**Part 8: Conclusion and Key Takeaways**
* [8.1. Summary of Our Journey](#part8-conclusion-pro)
* [8.2. Key Architectural Principles of Advanced RAG Systems](#part8-2-principles-pro-adv)
* [8.3. Future Directions](#part8-3-future-pro-adv)

## Part 1: Setting the Stage - Foundations and Our Core Challenge

### 1.1. Introduction: The Limits of "Shallow" RAG

Retrieval-Augmented Generation (RAG) has become the dominant paradigm for creating knowledge-intensive AI systems. The standard approach—a linear, three-step pipeline of **Retrieve -> Augment -> Generate**—is remarkably effective for simple, fact-based queries. However, this "shallow" RAG architecture reveals critical weaknesses when faced with complex questions that demand synthesis, comparison, and multi-step reasoning across a large and varied knowledge base.

The next frontier in RAG is not about bigger models or larger context windows, but about greater **autonomy and intelligence** in the retrieval and reasoning process. The industry is moving from static chains to dynamic, agentic systems that can emulate a human researcher's workflow. These systems can decompose complex problems, select appropriate tools, dynamically adapt their retrieval strategies, and critique their own progress.

In this comprehensive guide, we will build a powerful, **standalone** implementation of a **Deep Thinking RAG Pipeline**. We will meticulously engineer every component, from a sophisticated multi-stage, adaptive retrieval funnel to a tool-augmented, self-critiquing policy engine. We will begin by exposing the failure of a vanilla RAG system on a challenging query, and then, step-by-step, construct our advanced agent using **LangGraph** to orchestrate its complex, cyclical reasoning. By the end, you will have a production-grade framework and a deep, architectural understanding of how to build RAG systems that can truly *think*.

### 1.2. Environment Setup: API Keys, Imports, and Configuration

We begin by setting up our foundational components. This includes securely managing API keys, importing all necessary libraries, and defining a global configuration dictionary. We will use **LangSmith** for tracing, which is an indispensable tool for visualizing and debugging the complex, non-linear execution paths of our reasoning agent. For our new web search capability, we will also add the **Tavily AI** API key.

In [1]:
import os
import re
import json
from getpass import getpass
from pprint import pprint
import uuid
from typing import List, Dict, TypedDict, Literal, Optional

# Securely set API keys
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Enter your {var}: ")

#_set_env("OPENAI_API_KEY")
#_set_env("LANGSMITH_API_KEY")
#_set_env("TAVILY_API_KEY")


# Optional: For accessing SEC filings programmatically
# _set_env("SEC_API_KEY")

# Configure LangSmith tracing
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Advanced-Deep-Thinking-RAG-v2"

# Load API keys from environment variables or .env file
# Never hardcode API keys in notebooks! Use .env file or environment variables
from dotenv import load_dotenv
load_dotenv()  # Load from .env file if it exists

# Set API keys from environment (already loaded from .env or system env)
# If not set, use getpass to prompt securely
if not os.environ.get("LANGSMITH_API_KEY"):
    _set_env("LANGSMITH_API_KEY")
if not os.environ.get("TAVILY_API_KEY"):
    _set_env("TAVILY_API_KEY")

# Set Azure OpenAI specific environment variables
# Load from .env file or prompt if not set
if not os.environ.get("AZURE_OPENAI_API_KEY"):
    _set_env("AZURE_OPENAI_API_KEY")
if not os.environ.get("AZURE_OPENAI_ENDPOINT"):
    endpoint = input("Enter AZURE_OPENAI_ENDPOINT (or set in .env): ").strip()
    if endpoint:
        os.environ["AZURE_OPENAI_ENDPOINT"] = endpoint
if not os.environ.get("AZURE_OPENAI_API_VERSION"):
    os.environ["AZURE_OPENAI_API_VERSION"] = "2025-01-01-preview"
if not os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"):
    deployment = input("Enter AZURE_OPENAI_DEPLOYMENT_NAME (or set in .env): ").strip()
    if deployment:
        os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = deployment


# Central Configuration Dictionary
config = {
    "data_dir": "./data",
    "vector_store_dir": "./vector_store",
    "llm_provider": "azure_openai",
    "reasoning_llm": "gpt-3.5-turbo-0125", # Changed to a different LLM to avoid rate limits
    "fast_llm": "gpt-3.5-turbo-0125",
    "embedding_model": "text-embedding-3-small",
    "reranker_model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "max_reasoning_iterations": 7, # Maximum loops for the reasoning agent
    "top_k_retrieval": 10,       # Number of documents for initial broad recall
    "top_n_rerank": 3,
    "azure_deployment_name": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

# Create directories if they don't exist
os.makedirs(config["data_dir"], exist_ok=True)
os.makedirs(config["vector_store_dir"], exist_ok=True)

print("Environment and configuration set up successfully.")
pprint(config)


Environment and configuration set up successfully.
{'azure_api_version': '2025-01-01-preview',
 'azure_deployment_name': 'chatbottest01-llm-gpt-4o-mini',
 'azure_endpoint': 'https://manju-mh8ukqqk-eastus2.cognitiveservices.azure.com/openai/deployments/chatbottest01-llm-gpt-4o-mini/chat/completions?api-version=2025-01-01-preview',
 'data_dir': './data',
 'embedding_model': 'text-embedding-3-small',
 'fast_llm': 'gpt-3.5-turbo-0125',
 'llm_provider': 'azure_openai',
 'max_reasoning_iterations': 7,
 'reasoning_llm': 'gpt-3.5-turbo-0125',
 'reranker_model': 'cross-encoder/ms-marco-MiniLM-L-6-v2',
 'top_k_retrieval': 10,
 'top_n_rerank': 3,
 'vector_store_dir': './vector_store'}


### 1.3. The Dataset: Preparing Our Knowledge Base from Complex Documents

Our knowledge base will be the full text of NVIDIA's 2023 10-K filing. Instead of a dummy file, we will programmatically download the actual filing from the SEC's EDGAR database. This document is a dense, 100+ page report detailing their business, financials, and risks. This is a perfect test case because answering sophisticated questions requires connecting information spread across disparate sections like 'Business Overview', 'Risk Factors', and 'Management's Discussion and Analysis' (MD&A).

In [3]:
import os
from pathlib import Path
from langchain_core.documents import Document

# Ensure pypdf is installed for PDF loading
try:
    import pypdf
except ImportError:
    print("Installing pypdf package...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pypdf"])
    import pypdf
    print("✓ pypdf installed successfully")

from langchain_community.document_loaders import PyPDFLoader, TextLoader

def load_documents_from_data_folder(data_dir):
    """
    Load all documents from the data folder at the project root.
    Supports PDF and text files.
    """
    documents = []
    data_path = Path(data_dir)
    
    # Resolve to absolute path to ensure we're using the correct directory
    data_path = data_path.resolve()
    
    if not data_path.exists():
        raise ValueError(f"Data directory does not exist: {data_path}")
    
    print(f"Loading documents from: {data_path}")
    
    # Get all files in the data folder
    pdf_files = list(data_path.glob("*.pdf"))
    txt_files = list(data_path.glob("*.txt"))
    
    print(f"Found {len(pdf_files)} PDF file(s) and {len(txt_files)} text file(s)")
    
    # Load PDF files
    for pdf_file in pdf_files:
        print(f"\nLoading PDF: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            docs = loader.load()
            # Add source metadata to each document
            for doc in docs:
                doc.metadata['source'] = str(pdf_file)
                doc.metadata['file_name'] = pdf_file.name
            documents.extend(docs)
            print(f"  ✓ Loaded {len(docs)} pages from {pdf_file.name}")
        except Exception as e:
            print(f"  ✗ Error loading {pdf_file.name}: {e}")
    
    # Load text files
    for txt_file in txt_files:
        print(f"\nLoading text file: {txt_file.name}")
        try:
            loader = TextLoader(str(txt_file), encoding='utf-8')
            docs = loader.load()
            # Add source metadata to each document
            for doc in docs:
                doc.metadata['source'] = str(txt_file)
                doc.metadata['file_name'] = txt_file.name
            documents.extend(docs)
            print(f"  ✓ Loaded text from {txt_file.name}")
        except Exception as e:
            print(f"  ✗ Error loading {txt_file.name}: {e}")
    
    return documents

# Load all documents from the data folder (located at project root)
print("=" * 60)
print("Loading documents from data folder...")
print("=" * 60)
all_documents = load_documents_from_data_folder(config["data_dir"])

print(f"\n{'=' * 60}")
print(f"Total documents loaded: {len(all_documents)}")
print(f"{'=' * 60}")

if all_documents:
    print(f"\n--- Sample content from first document ---")
    print(f"Source: {all_documents[0].metadata.get('file_name', 'Unknown')}")
    print(f"Content preview (first 500 chars):")
    print("-" * 60)
    print(all_documents[0].page_content[:500] + "...")
    print("-" * 60)
else:
    print("\n⚠️  No documents were loaded. Please ensure there are PDF or text files in the data folder.")

Loading documents from data folder...
Loading documents from: /Users/pradeepkumar/Development/coding/agentic-ai-deep-rag/notebooks/data
Found 1 PDF file(s) and 0 text file(s)

Loading PDF: Copy of Benchmarking-Green-Hydrogen-in-Indias-Energy-Transition.pdf
  ✓ Loaded 30 pages from Copy of Benchmarking-Green-Hydrogen-in-Indias-Energy-Transition.pdf

Total documents loaded: 30

--- Sample content from first document ---
Source: Copy of Benchmarking-Green-Hydrogen-in-Indias-Energy-Transition.pdf
Content preview (first 500 chars):
------------------------------------------------------------
Citation: Dubey, B.; Agrawal, S.;
Sharma, A.K. India’s Renewable
Energy Portfolio: An Investigation of
the Untapped Potential of RE,
Policies, and Incentives Favoring
Energy Security in the Country.
Energies 2023, 16, 5491.
https://doi.org/10.3390/en16145491
Academic Editor: Manolis Souliotis
Received: 15 June 2023
Revised: 11 July 2023
Accepted: 17 July 2023
Published: 20 July 2023
Copyright: © 2023 by

### 1.4. The Upgraded Challenge: A Multi-Source, Multi-Hop Query We Will Conquer

This is the query designed to break our baseline RAG system and showcase the power of our advanced agent. It requires the agent to perform multiple distinct information retrieval steps from *different sources* (the static 10-K and the live web) and then synthesize the findings into a coherent analytical narrative.

> **The Query:** "Based on NVIDIA's 2023 10-K filing, identify their key risks related to competition. Then, find recent news (post-filing, from 2024) about AMD's AI chip strategy and explain how this new strategy directly addresses or exacerbates one of NVIDIA's stated risks."

## Part 2: The Baseline - Building and Breaking a "Vanilla" RAG Pipeline

### 2.1. Code Dependency: Document Loading and Naive Chunking Strategy

Our baseline pipeline begins with a standard approach: load the entire document and split it into fixed-size chunks using a `RecursiveCharacterTextSplitter`. This method is fast but semantically naive, often splitting paragraphs or related ideas across different chunks—a primary source of failure for complex queries.

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("Chunking the documents...")

# Use the documents loaded from the data folder in section 1.3
# Make sure you've run section 1.3 first to load all_documents
if 'all_documents' not in globals() or not all_documents:
    raise ValueError("Please run section 1.3 first to load documents from the data folder.")

# Split all documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
doc_chunks = text_splitter.split_documents(all_documents)

print(f"Documents loaded and split into {len(doc_chunks)} chunks.")
print(f"Total pages/documents processed: {len(all_documents)}")

Chunking the documents...
Documents loaded and split into 149 chunks.
Total pages/documents processed: 30


In [5]:
# To avoid re-running pip install every time, this is commented out if dependencies are already met.
# If you encounter ModuleNotFoundError, uncomment the line below and run this cell.
!pip install -U langchain langgraph langchain_openai chromadb beautifulsoup4 rank_bm25 lxml sentence-transformers ragas arxiv rich sec-api unstructured[html] tavily-python langchain-community

import os
import re
import json
from getpass import getpass
from pprint import pprint
import uuid
from typing import List, Dict, TypedDict, Literal, Optional

# Securely set API keys
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Enter your {var}: ")

_set_env("OPENAI_API_KEY")

# Load API keys from environment variables or .env file
# Never hardcode API keys! Use .env file or environment variables
from dotenv import load_dotenv
load_dotenv()  # Load from .env file if it exists

# Set API keys from environment (already loaded from .env or system env)
# If not set, use getpass to prompt securely
if not os.environ.get("LANGSMITH_API_KEY"):
    _set_env("LANGSMITH_API_KEY")
if not os.environ.get("TAVILY_API_KEY"):
    _set_env("TAVILY_API_KEY")
# Optional: For accessing SEC filings programmatically
# _set_env("SEC_API_KEY")

# Configure LangSmith tracing
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Advanced-Deep-Thinking-RAG-v2"

# Central Configuration Dictionary
config = {
    "data_dir": "./data",
    "vector_store_dir": "./vector_store",
    "llm_provider": "openai",
    "reasoning_llm": "gpt-4o-mini", # Changed to a faster LLM to avoid rate limits
    "fast_llm": "gpt-4o-mini",
    "embedding_model": "BAAI/bge-small-en-v1.5", # Changed to a local embedding model
    "reranker_model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "max_reasoning_iterations": 7, # Maximum loops for the reasoning agent
    "top_k_retrieval": 10,       # Number of documents for initial broad recall
    "top_n_rerank": 3,
}

# Create directories if they don't exist
os.makedirs(config["data_dir"], exist_ok=True)
os.makedirs(config["vector_store_dir"], exist_ok=True)

print("Environment and configuration set up successfully.")
pprint(config)


zsh:1: no matches found: unstructured[html]
Environment and configuration set up successfully.
{'data_dir': './data',
 'embedding_model': 'BAAI/bge-small-en-v1.5',
 'fast_llm': 'gpt-4o-mini',
 'llm_provider': 'openai',
 'max_reasoning_iterations': 7,
 'reasoning_llm': 'gpt-4o-mini',
 'reranker_model': 'cross-encoder/ms-marco-MiniLM-L-6-v2',
 'top_k_retrieval': 10,
 'top_n_rerank': 3,
 'vector_store_dir': './vector_store'}


In [6]:
import os
from pathlib import Path
from langchain_core.documents import Document

# Ensure pypdf is installed for PDF loading
try:
    import pypdf
except ImportError:
    print("Installing pypdf package...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pypdf"])
    import pypdf
    print("✓ pypdf installed successfully")

from langchain_community.document_loaders import PyPDFLoader, TextLoader

def load_documents_from_data_folder(data_dir):
    """
    Load all documents from the data folder at the project root.
    Supports PDF and text files.
    """
    documents = []
    data_path = Path(data_dir)
    
    # Resolve to absolute path to ensure we're using the correct directory
    data_path = data_path.resolve()
    
    if not data_path.exists():
        raise ValueError(f"Data directory does not exist: {data_path}")
    
    print(f"Loading documents from: {data_path}")
    
    # Get all files in the data folder
    pdf_files = list(data_path.glob("*.pdf"))
    txt_files = list(data_path.glob("*.txt"))
    
    print(f"Found {len(pdf_files)} PDF file(s) and {len(txt_files)} text file(s)")
    
    # Load PDF files
    for pdf_file in pdf_files:
        print(f"\nLoading PDF: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            docs = loader.load()
            # Add source metadata to each document
            for doc in docs:
                doc.metadata['source'] = str(pdf_file)
                doc.metadata['file_name'] = pdf_file.name
            documents.extend(docs)
            print(f"  ✓ Loaded {len(docs)} pages from {pdf_file.name}")
        except Exception as e:
            print(f"  ✗ Error loading {pdf_file.name}: {e}")
    
    # Load text files
    for txt_file in txt_files:
        print(f"\nLoading text file: {txt_file.name}")
        try:
            loader = TextLoader(str(txt_file), encoding='utf-8')
            docs = loader.load()
            # Add source metadata to each document
            for doc in docs:
                doc.metadata['source'] = str(txt_file)
                doc.metadata['file_name'] = txt_file.name
            documents.extend(docs)
            print(f"  ✓ Loaded text from {txt_file.name}")
        except Exception as e:
            print(f"  ✗ Error loading {txt_file.name}: {e}")
    
    return documents

# Load all documents from the data folder (located at project root)
print("=" * 60)
print("Loading and parsing documents from data folder...")
print("=" * 60)
all_documents = load_documents_from_data_folder(config["data_dir"])

print(f"\n{'=' * 60}")
print(f"Total documents loaded: {len(all_documents)}")
print(f"{'=' * 60}")

if all_documents:
    print(f"\n--- Sample content from first document ---")
    print(f"Source: {all_documents[0].metadata.get('file_name', 'Unknown')}")
    print(f"Content preview (first 500 chars):")
    print("-" * 60)
    print(all_documents[0].page_content[:500] + "...")
    print("-" * 60)
else:
    print("\n⚠️  No documents were loaded. Please ensure there are PDF or text files in the data folder.")

Loading and parsing documents from data folder...
Loading documents from: /Users/pradeepkumar/Development/coding/agentic-ai-deep-rag/notebooks/data
Found 1 PDF file(s) and 0 text file(s)

Loading PDF: Copy of Benchmarking-Green-Hydrogen-in-Indias-Energy-Transition.pdf
  ✓ Loaded 30 pages from Copy of Benchmarking-Green-Hydrogen-in-Indias-Energy-Transition.pdf

Total documents loaded: 30

--- Sample content from first document ---
Source: Copy of Benchmarking-Green-Hydrogen-in-Indias-Energy-Transition.pdf
Content preview (first 500 chars):
------------------------------------------------------------
Citation: Dubey, B.; Agrawal, S.;
Sharma, A.K. India’s Renewable
Energy Portfolio: An Investigation of
the Untapped Potential of RE,
Policies, and Incentives Favoring
Energy Security in the Country.
Energies 2023, 16, 5491.
https://doi.org/10.3390/en16145491
Academic Editor: Manolis Souliotis
Received: 15 June 2023
Revised: 11 July 2023
Accepted: 17 July 2023
Published: 20 July 2023
Copyrigh

### 2.1.5. Loading or Generating Embeddings (Optional)

You can use the embedding pipeline to generate embeddings once and reuse them for both basic and deep RAG. This is especially useful for server-side deployments.

**Option 1: Use Pre-Generated Embeddings (Recommended for Production)**
Run the embedding generation script first:
```bash
python scripts/generate_embeddings.py
```

Then load them in the notebook below.

**Option 2: Generate Embeddings in Notebook**
The notebook will generate embeddings on-the-fly if they don't exist.


In [ ]:
# Option: Load or generate embeddings using the embedding pipeline
# This allows you to reuse embeddings across multiple notebook runs

import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

try:
    from src.embedding_pipeline import load_or_generate_embeddings
    from src.embeddings import create_embedding_function
    from src.vector_store import create_retriever
    
    print("=" * 60)
    print("LOADING OR GENERATING EMBEDDINGS")
    print("=" * 60)
    
    # Try to load existing embeddings, or generate new ones
    # Set force_regenerate=True to always generate new embeddings
    use_pre_generated = True  # Set to False to always generate in notebook
    
    if use_pre_generated:
        print("\nAttempting to load pre-generated embeddings...")
        try:
            vector_store = load_or_generate_embeddings(
                config=config,
                force_regenerate=False
            )
            embedding_function = create_embedding_function(config)
            baseline_retriever = create_retriever(vector_store, k=3)
            
            # Get document chunks from vector store for compatibility
            # Note: We'll use the vector store directly, but we need doc_chunks for some operations
            print("\n✓ Using pre-generated embeddings!")
            print("Note: If you need doc_chunks variable, it will be created in the next cell.")
            
            # Set a flag to indicate we're using pre-generated embeddings
            using_pre_generated_embeddings = True
            
        except Exception as e:
            print(f"\n⚠ Could not load pre-generated embeddings: {e}")
            print("Will generate embeddings in the next cell...")
            using_pre_generated_embeddings = False
    else:
        using_pre_generated_embeddings = False
        print("\nSkipping pre-generated embeddings. Will generate in next cell.")
        
except ImportError as e:
    print(f"⚠ Could not import embedding pipeline: {e}")
    print("Will use inline embedding generation in the next cell...")
    using_pre_generated_embeddings = False


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("Chunking the documents...")

# Use the documents loaded from the data folder in section 1.3
# Make sure you've run section 1.3 first to load all_documents
if 'all_documents' not in globals() or not all_documents:
    raise ValueError("Please run section 1.3 first to load documents from the data folder.")

# Split all documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
doc_chunks = text_splitter.split_documents(all_documents)

print(f"Documents loaded and split into {len(doc_chunks)} chunks.")
print(f"Total pages/documents processed: {len(all_documents)}")

Chunking the documents...
Documents loaded and split into 149 chunks.
Total pages/documents processed: 30


In [ ]:
# Check if we're using pre-generated embeddings
if 'using_pre_generated_embeddings' in globals() and using_pre_generated_embeddings:
    print("Using pre-generated embeddings from previous cell.")
    print("Skipping vector store creation.")
    # doc_chunks will be needed for some operations, so we'll extract them if needed
    # For now, we'll create a placeholder - the actual chunks are in the vector store
    if 'doc_chunks' not in globals():
        print("Note: doc_chunks variable not set. Some operations may require it.")
else:
    # Use FAISS instead of ChromaDB to avoid compatibility issues
    from langchain_community.vectorstores import FAISS
    
    # Ensure required packages are installed
    try:
        import faiss
    except ImportError:
        print("Installing faiss-cpu...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "faiss-cpu"])
        import faiss
        print("✓ faiss-cpu installed successfully")
    
    # Create embedding function
    from src.embeddings import create_embedding_function
    from src.vector_store import create_vector_store, create_retriever
    
    print("Creating baseline vector store with FAISS...")
    embedding_function = create_embedding_function(config)
    
    baseline_vector_store = FAISS.from_documents(
        documents=doc_chunks,
        embedding=embedding_function
    )
    baseline_retriever = baseline_vector_store.as_retriever(search_kwargs={"k": 3})
    
    print(f"Vector store created with {len(doc_chunks)} documents.")

Creating baseline vector store with FAISS...


/var/folders/w5/k1crj13j33n3_y1sj0nlz5yw0000gn/T/ipykernel_7519/1555405908.py:28: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceBgeEmbeddings(


Vector store created with 149 documents.


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os

# Ensure langchain_openai is installed
try:
    from langchain_openai import AzureChatOpenAI, ChatOpenAI
except ImportError:
    print("Installing langchain_openai package...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "langchain-openai"])
    from langchain_openai import AzureChatOpenAI, ChatOpenAI
    print("✓ langchain_openai installed successfully")

# Check if using Azure OpenAI or regular OpenAI
if config.get("llm_provider") == "azure_openai":
    template = """You are an AI energy sector analyst specializing in renewable energy, green hydrogen, and energy transition. Answer the question based only on the following context from energy sector documents:

{context}

Question: {question}

Provide a clear, accurate answer based on the context provided. If the context doesn't contain enough information to answer the question, say so."""
    prompt = ChatPromptTemplate.from_template(template)
    
    # Get Azure credentials from environment or config
    api_key = os.environ.get("AZURE_OPENAI_API_KEY") or config.get("azure_api_key")
    azure_endpoint = config.get("azure_endpoint") or os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_deployment = config.get("azure_deployment_name") or config.get("fast_llm")
    api_version = config.get("azure_api_version") or os.environ.get("AZURE_OPENAI_API_VERSION", "2025-01-01-preview")
    
    if not api_key:
        raise ValueError("Azure OpenAI API key not found. Please set AZURE_OPENAI_API_KEY environment variable or config['azure_api_key']")
    if not azure_endpoint:
        raise ValueError("Azure OpenAI endpoint not found. Please set config['azure_endpoint'] or AZURE_OPENAI_ENDPOINT environment variable")
    
    llm = AzureChatOpenAI(
        azure_deployment=azure_deployment,
        azure_endpoint=azure_endpoint,
        api_version=api_version,
        api_key=api_key,
        temperature=0
    )
    print("Using Azure OpenAI...")
else:
    template = """You are an AI energy sector analyst specializing in renewable energy, green hydrogen, and energy transition. Answer the question based only on the following context from energy sector documents:

{context}

Question: {question}

Provide a clear, accurate answer based on the context provided. If the context doesn't contain enough information to answer the question, say so."""
    prompt = ChatPromptTemplate.from_template(template)
    
    llm = ChatOpenAI(model=config["fast_llm"], temperature=0)
    print("Using OpenAI...")

def format_docs(docs):
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

baseline_rag_chain = (
    {"context": baseline_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("Baseline RAG chain assembled successfully for energy sector documents.")

Using OpenAI...
Baseline RAG chain assembled successfully for energy sector documents.


In [10]:
import os
from getpass import getpass
from pprint import pprint

# Securely set API keys
def _set_env(var: str):
    if var not in os.environ:
        os.environ[var] = getpass(f"Enter {var}: ")

# Set Azure OpenAI environment variables (adjust these to your actual values)
# Uncomment and set these if not already set in your environment
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key-here"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-endpoint.openai.azure.com/"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2025-01-01-preview"
# os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "your-deployment-name"

# Configure LangSmith tracing (optional)
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_PROJECT"] = "Deep-Thinking-RAG"

# Central Configuration Dictionary
config = {
    "data_dir": "./data",  # Use project root data folder
    "vector_store_dir": "./vector_store",  # Use project root for vector store
    "llm_provider": "azure_openai",
    "reasoning_llm": "gpt-3.5-turbo-0125",  # This will be the model deployed under AZURE_OPENAI_DEPLOYMENT_NAME
    "fast_llm": "gpt-3.5-turbo-0125",  # This will be the model deployed under AZURE_OPENAI_DEPLOYMENT_NAME
    "embedding_model": "BAAI/bge-small-en-v1.5",  # Using HuggingFace model to avoid Azure embedding issues
    "reranker_model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "max_reasoning_iterations": 7,  # Maximum loops for the reasoning agent
    "top_k_retrieval": 10,  # Number of documents for initial broad recall
    "top_n_rerank": 3,
    "azure_deployment_name": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_api_version": os.environ.get("AZURE_OPENAI_API_VERSION", "2025-01-01-preview"),
}

# Create directories if they don't exist (only if they're in writable locations)
try:
    os.makedirs(config["data_dir"], exist_ok=True)
except (OSError, PermissionError) as e:
    print(f"⚠ Warning: Could not create data directory: {e}")
    print(f"  Using existing directory: {config['data_dir']}")

try:
    os.makedirs(config["vector_store_dir"], exist_ok=True)
except (OSError, PermissionError) as e:
    print(f"⚠ Warning: Could not create vector store directory: {e}")
    print(f"  Using existing directory: {config['vector_store_dir']}")

print("Environment and configuration set up successfully.")
pprint(config)

Environment and configuration set up successfully.
{'azure_api_version': '2025-01-01-preview',
 'azure_deployment_name': 'chatbottest01-llm-gpt-4o-mini',
 'azure_endpoint': 'https://manju-mh8ukqqk-eastus2.cognitiveservices.azure.com/openai/deployments/chatbottest01-llm-gpt-4o-mini/chat/completions?api-version=2025-01-01-preview',
 'data_dir': './data',
 'embedding_model': 'BAAI/bge-small-en-v1.5',
 'fast_llm': 'gpt-3.5-turbo-0125',
 'llm_provider': 'azure_openai',
 'max_reasoning_iterations': 7,
 'reasoning_llm': 'gpt-3.5-turbo-0125',
 'reranker_model': 'cross-encoder/ms-marco-MiniLM-L-6-v2',
 'top_k_retrieval': 10,
 'top_n_rerank': 3,
 'vector_store_dir': './vector_store'}


### 2.2. Code Dependency: Creating the Vector Store with Dense Embeddings

Next, we embed these chunks using OpenAI's `text-embedding-3-small` model and index them in a ChromaDB vector store. This store will power our baseline retriever, which performs a simple semantic similarity search.

In [11]:
# Use FAISS instead of ChromaDB to avoid compatibility issues
from langchain_community.vectorstores import FAISS

# Ensure required packages are installed
try:
    import faiss
except ImportError:
    print("Installing faiss-cpu...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faiss-cpu"])
    import faiss
    print("✓ faiss-cpu installed successfully")

# Determine which embedding class to use based on the model name
embedding_model = config['embedding_model']
embedding_function = None

if embedding_model.startswith('text-embedding') or 'openai' in embedding_model.lower():
    # Try OpenAI embeddings (Azure or regular)
    if config.get("llm_provider") == "azure_openai":
        try:
            from langchain_openai import AzureOpenAIEmbeddings
            import os
            
            api_key = os.environ.get("AZURE_OPENAI_API_KEY") or config.get("azure_api_key")
            azure_endpoint = config.get("azure_endpoint") or os.environ.get("AZURE_OPENAI_ENDPOINT")
            api_version = config.get("azure_api_version") or os.environ.get("AZURE_OPENAI_API_VERSION", "2025-01-01-preview")
            
            # Try to use Azure OpenAI embeddings
            # Note: You may need a separate deployment for embeddings
            # If embedding deployment name is different, set it in config as 'azure_embedding_deployment'
            embedding_deployment = config.get("azure_embedding_deployment") or embedding_model
            
            if api_key and azure_endpoint:
                print("Attempting to use Azure OpenAI embeddings...")
                embedding_function = AzureOpenAIEmbeddings(
                    azure_deployment=embedding_deployment,
                    azure_endpoint=azure_endpoint,
                    api_version=api_version,
                    api_key=api_key
                )
                # Test the embedding function
                try:
                    test_embedding = embedding_function.embed_query("test")
                    print("✓ Azure OpenAI embeddings working")
                except Exception as e:
                    print(f"⚠ Azure OpenAI embeddings failed: {e}")
                    print("Falling back to HuggingFace embeddings...")
                    embedding_function = None
            else:
                print("⚠ Azure OpenAI credentials not found, falling back to HuggingFace embeddings...")
                embedding_function = None
        except ImportError:
            print("⚠ langchain-openai not available, falling back to HuggingFace embeddings...")
            embedding_function = None
        except Exception as e:
            print(f"⚠ Error setting up Azure OpenAI embeddings: {e}")
            print("Falling back to HuggingFace embeddings...")
            embedding_function = None
    
    # If Azure failed or not configured, try regular OpenAI
    if embedding_function is None and config.get("llm_provider") != "azure_openai":
        try:
            from langchain_openai import OpenAIEmbeddings
            print("Using OpenAI embeddings...")
            embedding_function = OpenAIEmbeddings(model=embedding_model)
        except ImportError:
            print("⚠ langchain-openai not available, falling back to HuggingFace embeddings...")
            embedding_function = None
        except Exception as e:
            print(f"⚠ Error setting up OpenAI embeddings: {e}")
            print("Falling back to HuggingFace embeddings...")
            embedding_function = None

# Fallback to HuggingFace embeddings if OpenAI didn't work
if embedding_function is None:
    try:
        import sentence_transformers
    except ImportError:
        print("Installing sentence-transformers...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "sentence-transformers"])
        import sentence_transformers
        print("✓ sentence-transformers installed")
    
    from langchain_community.embeddings import HuggingFaceBgeEmbeddings
    print("Using HuggingFace embeddings (fallback)...")
    
    # Use a reliable HuggingFace model if the config model isn't a valid HF model
    if embedding_model.startswith('text-embedding'):
        # If config has OpenAI model name, use a default HuggingFace model
        hf_model = "BAAI/bge-small-en-v1.5"
        print(f"  Using {hf_model} instead of {embedding_model}")
    else:
        hf_model = embedding_model
    
    embedding_function = HuggingFaceBgeEmbeddings(
        model_name=hf_model,
        encode_kwargs={'normalize_embeddings': True}
    )

print("Creating baseline vector store with FAISS...")
baseline_vector_store = FAISS.from_documents(
    documents=doc_chunks,
    embedding=embedding_function
)
baseline_retriever = baseline_vector_store.as_retriever(search_kwargs={"k": 3})

print(f"Vector store created with {len(doc_chunks)} documents.")

Using HuggingFace embeddings (fallback)...
Creating baseline vector store with FAISS...
Vector store created with 149 documents.


### 2.3. Code Dependency: Assembling the Simple RAG Chain

We use the LangChain Expression Language (LCEL) to construct our linear pipeline. The `RunnablePassthrough` allows us to pass the original question alongside the retrieved context into the prompt.

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI # Changed import
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


template = """You are an AI energy sector analyst specializing in renewable energy, green hydrogen, and energy transition. Answer the question based only on the following context from energy sector documents:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
llm = AzureChatOpenAI(
    azure_deployment=config["azure_deployment_name"],
    azure_endpoint=config["azure_endpoint"],
    api_version=config["azure_api_version"],
    temperature=0
)

def format_docs(docs):
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

baseline_rag_chain = (
    {"context": baseline_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("Baseline RAG chain assembled successfully.")


Baseline RAG chain assembled successfully.


### 2.4. The Critical Failure Case: Demonstrating the Need for Advanced Techniques

Now we execute our multi-source query against the baseline system. The retriever will attempt to find chunks that match the 'average' semantic meaning of the entire query. This will fail spectacularly because critical information (about AMD's 2024 strategy) does not exist in its knowledge base (the 2023 10-K).

In [13]:
# Try to import rich, install if missing
try:
    from rich.console import Console
    from rich.markdown import Markdown
    console = Console()
    use_rich = True
except ImportError:
    print("Installing rich package...")
    import subprocess
    import sys
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "rich"])
        from rich.console import Console
        from rich.markdown import Markdown
        console = Console()
        use_rich = True
        print("✓ rich installed successfully")
    except Exception as e:
        print(f"⚠ Could not install rich: {e}")
        print("Using standard print instead...")
        use_rich = False

# Validate Azure OpenAI credentials before running query
import os

if config.get("llm_provider") == "azure_openai":
    api_key = os.environ.get("AZURE_OPENAI_API_KEY") or config.get("azure_api_key")
    azure_endpoint = config.get("azure_endpoint") or os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_deployment = config.get("azure_deployment_name") or config.get("fast_llm")
    
    print("Checking Azure OpenAI configuration...")
    print(f"  Endpoint: {azure_endpoint}")
    print(f"  Deployment: {azure_deployment}")
    print(f"  API Key: {'***' + api_key[-4:] if api_key else 'NOT SET'}")
    
    if not api_key:
        raise ValueError(
            "❌ AZURE_OPENAI_API_KEY is not set. Please set it in your environment or config.\n"
            "   You can set it by running: os.environ['AZURE_OPENAI_API_KEY'] = 'your-key-here'"
        )
    if not azure_endpoint:
        raise ValueError(
            "❌ AZURE_OPENAI_ENDPOINT is not set. Please set it in your environment or config.\n"
            "   You can set it by running: os.environ['AZURE_OPENAI_ENDPOINT'] = 'your-endpoint-here'"
        )
    
    # Test the LLM connection
    print("\nTesting Azure OpenAI connection...")
    try:
        test_response = llm.invoke("Hello")
        print("✓ Azure OpenAI connection successful")
    except Exception as e:
        print(f"❌ Azure OpenAI connection failed: {e}")
        print("\nPlease check:")
        print("  1. Your API key is correct and active")
        print("  2. Your endpoint URL is correct")
        print("  3. Your deployment name exists and is active")
        print("  4. Your subscription has sufficient quota")
        raise

# Query specific to the green hydrogen benchmarking document
complex_query_adv = "What are the key cost benchmarks and performance metrics for green hydrogen production in India as outlined in the benchmarking document? Compare these to conventional hydrogen production methods and identify the main factors affecting cost competitiveness."

print("\n" + "=" * 60)
print("Executing complex query on the baseline RAG chain...")
print("=" * 60)
print(f"Query: {complex_query_adv}")
print("=" * 60 + "\n")

try:
    baseline_result = baseline_rag_chain.invoke(complex_query_adv)
    
    if use_rich:
        console.print("\n--- BASELINE RAG OUTPUT ---")
        console.print(Markdown(baseline_result))
    else:
        print("\n" + "=" * 60)
        print("BASELINE RAG OUTPUT")
        print("=" * 60)
        print(baseline_result)
        print("=" * 60)
except Exception as e:
    print(f"\n❌ Error executing query: {e}")
    print("\nTroubleshooting steps:")
    print("1. Verify your Azure OpenAI credentials are set correctly")
    print("2. Check that your deployment name matches your Azure resource")
    print("3. Ensure your subscription is active and has quota")
    print("4. Verify the endpoint URL is correct")
    raise

Checking Azure OpenAI configuration...
  Endpoint: https://manju-mh8ukqqk-eastus2.cognitiveservices.azure.com/openai/deployments/chatbottest01-llm-gpt-4o-mini/chat/completions?api-version=2025-01-01-preview
  Deployment: chatbottest01-llm-gpt-4o-mini
  API Key: ***NRA1

Testing Azure OpenAI connection...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✓ Azure OpenAI connection successful

Executing complex query on the baseline RAG chain...
Query: What are the key cost benchmarks and performance metrics for green hydrogen production in India as outlined in the benchmarking document? Compare these to conventional hydrogen production methods and identify the main factors affecting cost competitiveness.



--- BASELINE RAG OUTPUT ---

The provided context does not contain specific information regarding the cost benchmarks and performance metrics   
for green hydrogen production in India, nor does it compare these to conventional hydrogen production methods. The 
text primarily discusses the challenges and potential of renewable energy sources, particularly solar energy, in   
India, along with the government's policies and the importance of clean energy for economic and climatic growth.   

To answer your question accurately, I would need specific details from a benchmarking document that outlines the   
cost benchmarks and performance metrics for green hydrogen production, as well as comparisons to conventional      
methods and factors affecting cost competitiveness. If you have such information or another context, please provide
it, and I would be happy to assist!

In [14]:
from rich.console import Console
from rich.markdown import Markdown

console = Console()

complex_query_adv = "What are the key cost benchmarks and performance metrics for green hydrogen production in India as outlined in the benchmarking document? Compare these to conventional hydrogen production methods and identify the main factors affecting cost competitiveness."

print("Executing complex query on the baseline RAG chain...")
baseline_result = baseline_rag_chain.invoke(complex_query_adv)

console.print("--- BASELINE RAG FAILED OUTPUT ---")
console.print(Markdown(baseline_result))

Executing complex query on the baseline RAG chain...


--- BASELINE RAG FAILED OUTPUT ---

The provided context does not contain specific information regarding the cost benchmarks and performance metrics   
for green hydrogen production in India, nor does it compare these to conventional hydrogen production methods. The 
text primarily discusses the challenges and potential of renewable energy sources, particularly solar energy, in   
India, along with the government's policies and the importance of clean energy for economic and climatic growth.   

To answer your question accurately, I would need specific details from a benchmarking document that outlines the   
cost benchmarks and performance metrics for green hydrogen production, as well as comparisons to conventional      
methods and the factors affecting cost competitiveness. If you have access to such a document or additional        
information, please provide it, and I would be happy to assist you further.

### 2.5. Diagnosis: Why Did It Fail?

The output is a classic failure case for RAG systems confined to a static knowledge base.

1.  **Irrelevant Context:** The retriever, trying to satisfy all parts of the query at once, likely pulled chunks related to "competition" and "AMD" from the 10-K, but this information is general and lacks the specifics required.
2.  **Missing Information:** The 2023 filing **cannot** contain information about events in 2024. The baseline system has no mechanism to access external, up-to-date knowledge.
3.  **No Synthesis:** The system correctly states that it lacks the required information. It cannot perform the requested synthesis because it failed to retrieve one of the two necessary pieces of evidence. It lacks any mechanism to recognize this gap and use a different tool (like web search) to fill it.

## Part 3: The "Deep Thinking" Upgrade: Engineering an Autonomous Reasoning Engine

### 3.1. Code Dependency: Defining the `RAGState` - The Central Nervous System of Our Agent

To build our reasoning agent, we first need a robust way to manage its state. The `RAGState` `TypedDict` will serve as the central nervous system for our agent. It will be passed between every node in our LangGraph workflow, allowing the agent to maintain a coherent line of reasoning, track its progress, and build a comprehensive base of evidence over multiple steps. We will now enhance our `Step` Pydantic model to include a `tool` field, which will be crucial for routing.

In [15]:
from langchain_core.documents import Document
from pydantic import BaseModel, Field # Changed import
from typing import List, Dict, TypedDict, Literal, Optional

# Pydantic model for a single step in the reasoning plan
# Update Step model to use correct tool names
class Step(BaseModel):
    sub_question: str = Field(description="A clear, specific sub-question to answer.")
    justification: str = Field(description="Why this step is necessary.")
    tool: Literal["search_documents", "search_web"] = Field(
        description="The tool to use: 'search_documents' for local documents, 'search_web' for web search."
    )
    keywords: List[str] = Field(description="A list of critical keywords for searching.")
    document_section: Optional[str] = Field(
        default=None,
        description="Optional document section to filter search (for 'search_documents' only)."
    )

# Pydantic model for the overall plan
class Plan(BaseModel):
    steps: List[Step] = Field(description="A detailed, multi-step plan to answer the user's query.")

# TypedDict for storing the results of a completed step
class PastStep(TypedDict):
    step_index: int
    sub_question: str
    retrieved_docs: List[Document]
    summary: str

# The main state dictionary that will flow through the graph
class RAGState(TypedDict):
    original_question: str
    plan: Plan
    past_steps: List[PastStep]
    current_step_index: int
    retrieved_docs: List[Document]
    reranked_docs: List[Document]
    synthesized_context: str
    final_answer: str

print("RAGState and supporting Pydantic classes defined successfully. ") # Added a space to force modification


RAGState and supporting Pydantic classes defined successfully. 


### 3.2. Component 1: Dynamic Planning and Query Formulation

#### 3.2.1. The Tool-Aware Planner Agent: Decomposing the user query and selecting the right tool for each step.

The first cognitive act of our agent is to **plan**. We upgrade our 'Planner Agent' to be **tool-aware**. Its sole responsibility is to take the complex user query and decompose it into a structured, multi-step `Plan` object. Crucially, for each step, it must now decide whether the information is likely to be in the static document (`search_10k`) or requires up-to-date, external information (`search_web`). This decision-making at the planning stage is fundamental to the agent's intelligence.

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from rich.pretty import pprint as rprint

planner_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert research planner specializing in energy sector analysis. Your task is to create a clear, multi-step plan to answer a complex user query by retrieving information from multiple sources.

You have two tools available:
1. `search_documents`: Use this to search for information within the green hydrogen benchmarking document and other energy sector documents in the knowledge base. This is best for:
   - Technical specifications, cost benchmarks, and performance metrics
   - Policy frameworks, regulatory measures, and government initiatives
   - Market analysis, projections, and roadmaps
   - Comparative analysis with other countries or regions
   - Challenges, opportunities, and recommendations
   - Any information contained in the uploaded energy sector documents

2. `search_web`: Use this to search the public internet for:
   - Recent news and developments (post-document publication)
   - Current market trends and updates
   - Latest policy announcements or regulatory changes
   - Competitor information or industry developments
   - Any information not available in the local document knowledge base

Decompose the user's query into a series of simple, sequential sub-questions. For each step, decide which tool is more appropriate.

For `search_documents` steps, identify the most relevant sections or topics to search for (e.g., 'cost benchmarks', 'policy frameworks', 'technical specifications', 'market projections', 'challenges and opportunities', 'comparative analysis').

Ensure the plan is concise, with no more than 4-5 steps, focusing on the most critical information gathering and synthesis needed to comprehensively answer the user's query about green hydrogen and India's energy transition."""),
    ("human", "User Query: {question}")
])

reasoning_llm = AzureChatOpenAI(
    azure_deployment=config["azure_deployment_name"],
    azure_endpoint=config["azure_endpoint"],
    api_version=config["azure_api_version"],
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    temperature=0
)
planner_agent = planner_prompt | reasoning_llm.with_structured_output(Plan)
print("Tool-Aware Planner Agent created successfully.")

# Test the planner agent
print("--- Testing Planner Agent ---")
try:
    test_plan = planner_agent.invoke({"question": complex_query_adv})
    rprint(test_plan)
except Exception as e:
    print(f"Error during planner agent test: {e}")

Tool-Aware Planner Agent created successfully.
--- Testing Planner Agent ---


Plan(
│   steps=[
│   │   Step(
│   │   │   sub_question='What are the key cost benchmarks for green hydrogen production in India?',
│   │   │   justification='Understanding the cost benchmarks is essential to evaluate the economic viability of green hydrogen production in India.',
│   │   │   tool='search_documents',
│   │   │   keywords=['cost benchmarks', 'green hydrogen production', 'India'],
│   │   │   document_section='cost benchmarks'
│   │   ),
│   │   Step(
│   │   │   sub_question='What are the performance metrics for green hydrogen production in India?',
│   │   │   justification='Performance metrics will provide insights into the efficiency and effectiveness of green hydrogen production processes.',
│   │   │   tool='search_documents',
│   │   │   keywords=['performance metrics', 'green hydrogen production', 'India'],
│   │   │   document_section='performance metrics'
│   │   ),
│   │   Step(
│   │   │   sub_question='How do the cost benchmarks for green hydrogen compare to conventional hydrogen production methods?',
│   │   │   justification='A comparative analysis will highlight the differences in cost structures between green and conventional hydrogen production.',
│   │   │   tool='search_documents',
│   │   │   keywords=['cost comparison', 'green hydrogen', 'conventional hydrogen'],
│   │   │   document_section='comparative analysis'
│   │   ),
│   │   Step(
│   │   │   sub_question='What are the main factors affecting the cost competitiveness of green hydrogen production?',
│   │   │   justification='Identifying these factors will help understand the challenges and opportunities for green hydrogen in the market.',
│   │   │   tool='search_documents',
│   │   │   keywords=['cost competitiveness', 'green hydrogen', 'factors affecting cost'],
│   │   │   document_section='challenges and opportunities'
│   │   )
│   ]
)

#### 3.2.2. Query Rewriting and Expansion: Using an LLM to transform naive sub-questions into high-quality search queries.

A sub-question from the plan (e.g., "What are the risks?") might not be the optimal query for a vector database or web search engine. We create a 'Query Rewriter' agent that enriches the sub-question with keywords from the plan and context from previous steps, making it a much more effective search query.

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import AzureChatOpenAI
import os

query_rewriter_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a search query optimization expert specializing in energy sector and technical documents. Your task is to rewrite a given sub-question into a highly effective search query for a vector database or web search engine, using keywords and context from the research plan.

The rewritten query should be:
- Specific and use terminology likely to be found in energy sector documents (e.g., "green hydrogen", "levelized cost of hydrogen (LCOH)", "electrolyzer efficiency", "renewable energy", "energy transition", "policy frameworks", "cost benchmarks", "market projections")
- Structured to retrieve the most relevant text snippets from benchmarking documents, technical reports, or policy papers
- Use domain-specific terms that appear in green hydrogen and energy transition literature
- Include relevant metrics, comparisons, or technical specifications when applicable"""),
    ("human", "Current sub-question: {sub_question}\n\nRelevant keywords from plan: {keywords}\n\nContext from past steps:\n{past_context}")
])

# Ensure reasoning_llm is defined (should be from previous cell)
if 'reasoning_llm' not in locals():
    reasoning_llm = AzureChatOpenAI(
        azure_deployment=config["azure_deployment_name"],
        azure_endpoint=config["azure_endpoint"],
        api_version=config["azure_api_version"],
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
        temperature=0
    )

query_rewriter_agent = query_rewriter_prompt | reasoning_llm | StrOutputParser()
print("Query Rewriter Agent created successfully.")

# Test the rewriter agent
print("--- Testing Query Rewriter Agent ---")
try:
    # Ensure test_plan is available from the previous cell's execution
    if 'test_plan' in locals():
        # Use the first step from the plan for testing
        test_sub_q = test_plan.steps[0] if test_plan.steps else None
        if test_sub_q:
            # Updated test context for green hydrogen document
            test_past_context = """Step 1 Summary: The green hydrogen benchmarking document identifies key cost factors including electrolyzer costs, renewable energy prices, and infrastructure requirements. Step 2 Summary: India's green hydrogen policy framework includes production-linked incentives and targets for 5 million tonnes annual production by 2030."""
            rewritten_q = query_rewriter_agent.invoke({
                "sub_question": test_sub_q.sub_question,
                "keywords": test_sub_q.keywords if hasattr(test_sub_q, 'keywords') else "",
                "past_context": test_past_context
            })
            print(f"Original sub-question: {test_sub_q.sub_question}")
            print(f"Rewritten Search Query: {rewritten_q}")
        else:
            print("No steps found in test_plan.")
    else:
        # Create a sample test if test_plan is not available
        print("test_plan not defined. Running sample test...")
        sample_sub_question = "What are the cost benchmarks for green hydrogen production in India?"
        sample_keywords = "cost benchmarks, green hydrogen, India, production costs, LCOH"
        sample_context = "Previous analysis identified key cost factors including electrolyzer efficiency and renewable energy pricing."
        
        rewritten_q = query_rewriter_agent.invoke({
            "sub_question": sample_sub_question,
            "keywords": sample_keywords,
            "past_context": sample_context
        })
        print(f"Sample sub-question: {sample_sub_question}")
        print(f"Rewritten Search Query: {rewritten_q}")
except Exception as e:
    print(f"Error during query rewriter agent test: {e}")
    import traceback
    traceback.print_exc()

Query Rewriter Agent created successfully.
--- Testing Query Rewriter Agent ---
Original sub-question: What are the key cost benchmarks for green hydrogen production in India?
Rewritten Search Query: Rewritten search query: "key cost benchmarks for green hydrogen production in India 2023 electrolyzer costs renewable energy prices infrastructure requirements production-linked incentives annual targets 5 million tonnes"


#### 3.2.3. Entity and Constraint Extraction: Identifying metadata filters to enable filtered vector search.

This is a crucial step for precision when using the `search_10k` tool. Our planner already extracts the likely `document_section`. To use this, we need to re-process our documents, adding this section title as metadata to each chunk. This allows us to perform a *filtered search*, telling the vector store to *only* search within chunks that have the correct metadata (e.g., only search for risks in the 'Risk Factors' section).

In [18]:
import re
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rich.pretty import pprint as rprint
import uuid
from typing import List, Dict, TypedDict, Literal, Optional

print("Processing documents and adding metadata...")

# Use all_documents from section 1.3, or doc_chunks from section 2.1
if 'all_documents' in globals() and all_documents:
    source_documents = all_documents
    print(f"Using {len(source_documents)} documents from all_documents")
elif 'doc_chunks' in globals() and doc_chunks:
    source_documents = doc_chunks
    print(f"Using {len(doc_chunks)} chunks from doc_chunks")
else:
    raise ValueError("No documents found. Please run section 1.3 to load documents first.")

# Combine all document content for section extraction
raw_text = "\n\n".join([doc.page_content for doc in source_documents])

# Define patterns to identify section headers in energy sector documents
# Common patterns: numbered sections, chapter titles, major headings
section_patterns = [
    re.compile(r'^(Chapter\s+\d+[\.:]?\s+[A-Z][^\n]*)', re.MULTILINE | re.IGNORECASE),
    re.compile(r'^(\d+\.\s+[A-Z][^\n]*)', re.MULTILINE),  # Numbered sections like "1. Introduction"
    re.compile(r'^([A-Z][A-Z\s]{10,}\n)', re.MULTILINE),  # ALL CAPS headings
    re.compile(r'^([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\s*:?\s*\n)', re.MULTILINE),  # Title Case headings
]

# Try to find sections using multiple patterns
sections_data = []
current_section = ("Introduction", "")
lines = raw_text.split('\n')

# Simple heuristic: Look for lines that look like section headers
# (short lines, title case, followed by content)
potential_sections = []
for i, line in enumerate(lines):
    line_stripped = line.strip()
    # Skip empty lines
    if not line_stripped:
        continue
    
    # Check if line looks like a section header
    # Criteria: relatively short, title case or all caps, possibly numbered
    is_potential_header = (
        len(line_stripped) < 100 and  # Not too long
        (line_stripped[0].isupper() or line_stripped[0].isdigit()) and  # Starts with capital or number
        (line_stripped.isupper() or  # All caps
         any(c.isupper() for c in line_stripped if c.isalpha())) and  # Has capitals
        not line_stripped.endswith('.') or len(line_stripped.split()) < 10  # Not a full sentence
    )
    
    if is_potential_header:
        # Check if next few lines have content (not just another header)
        next_content = ""
        for j in range(i+1, min(i+5, len(lines))):
            if lines[j].strip() and not lines[j].strip()[0].isupper() or len(lines[j].strip()) > 50:
                next_content = lines[j].strip()
                break
        
        if next_content or i == 0:  # Include first potential header or ones with content
            potential_sections.append((i, line_stripped))

# If we found potential sections, use them
if potential_sections:
    for idx, (line_num, header) in enumerate(potential_sections):
        next_line_num = potential_sections[idx + 1][0] if idx + 1 < len(potential_sections) else len(lines)
        content = '\n'.join(lines[line_num+1:next_line_num]).strip()
        if content or idx == 0:  # Include first section even if empty
            sections_data.append((header, content))
else:
    # Fallback: Split by major paragraph breaks or use document structure
    # Split by double newlines (common in PDFs)
    paragraphs = raw_text.split('\n\n')
    current_section_title = "Document Content"
    current_content = []
    
    for para in paragraphs:
        para_stripped = para.strip()
        if not para_stripped:
            continue
        
        # Check if paragraph looks like a header
        if len(para_stripped) < 100 and para_stripped[0].isupper():
            # Save previous section if it has content
            if current_content:
                sections_data.append((current_section_title, '\n\n'.join(current_content)))
            current_section_title = para_stripped
            current_content = []
        else:
            current_content.append(para_stripped)
    
    # Add final section
    if current_content:
        sections_data.append((current_section_title, '\n\n'.join(current_content)))

# If still no sections, create one large section
if not sections_data:
    sections_data = [("Full Document", raw_text)]

print(f"Extracted {len(sections_data)} content sections with titles.")
print("Section titles found:")
for title, _ in sections_data[:10]:  # Show first 10
    print(f"  - {title[:80]}")

# Initialize text splitter if not already defined
if 'text_splitter' not in globals():
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

doc_chunks_with_metadata = []
for section_title, content in sections_data:
    # Skip sections with no content
    if not content.strip():
        continue

    # Clean section title for metadata
    clean_section_title = section_title.replace('\n', ' ').strip()[:200]  # Limit length

    # Get source file name from first document
    source_file = source_documents[0].metadata.get('file_name', 'unknown') if source_documents else 'unknown'

    section_chunks = text_splitter.split_text(content)
    if not section_chunks:  # If content is too small to chunk
        doc_chunks_with_metadata.append(
            Document(
                page_content=content.strip() or clean_section_title,
                metadata={
                    "section": clean_section_title,
                    "source_doc": source_file,
                    "file_name": source_file,
                    "id": str(uuid.uuid4())
                }
            )
        )
    else:
        for chunk in section_chunks:
            chunk_id = str(uuid.uuid4())
            doc_chunks_with_metadata.append(
                Document(
                    page_content=chunk,
                    metadata={
                        "section": clean_section_title,
                        "source_doc": source_file,
                        "file_name": source_file,
                        "id": chunk_id
                    }
                )
            )

print(f"\nCreated {len(doc_chunks_with_metadata)} chunks with section metadata.")

# Show sample chunks
print("\n--- Sample Chunks with Metadata ---")
for i, chunk in enumerate(doc_chunks_with_metadata[:3]):
    print(f"\nChunk {i+1}:")
    print(f"  Section: {chunk.metadata.get('section', 'Unknown')}")
    print(f"  Source: {chunk.metadata.get('file_name', 'Unknown')}")
    print(f"  Content preview: {chunk.page_content[:150]}...")

Processing documents and adding metadata...
Using 30 documents from all_documents
Extracted 301 content sections with titles.
Section titles found:
  - Citation: Dubey, B.; Agrawal, S.;
  - Energies 2023, 16, 5491.
  - , Seema Agrawal and Ashok Kumar Sharma
  - * Correspondence: bharatdubey8888@gmail.com
  - 482 GW of installed capacity and more than 40 percent of power production (inclu
  - 500 GW of green and clean energy by 2030. This paper highlights the important de
  - India has signiﬁcant potential for clean and green energy opportunities in the f
  - The MNRE estimates a renewable energy potential of around 1700.68 GW from abunda
  - Energies 2023, 16, 5491 2 of 30
  - A total of 42.6% of the nation’s installed capacity is made up of generation usi

Created 305 chunks with section metadata.

--- Sample Chunks with Metadata ---

Chunk 1:
  Section: Energies 2023, 16, 5491.
  Source: Copy of Benchmarking-Green-Hydrogen-in-Indias-Energy-Transition.pdf
  Content preview: https://do

### 3.3. Component 2: The Multi-Stage, Adaptive Retrieval Funnel

#### 3.3.1. NEW: The Retrieval Supervisor Agent

This is a new, crucial component for intelligent retrieval. Not all questions are created equal. Some benefit from semantic search (e.g., "What are the company's feelings on climate change?"), while others are better with keyword search (e.g., "What was the revenue for the 'Compute & Networking' segment?").

The **Retrieval Supervisor** is a small LLM agent that acts as a router. For each `search_10k` step, it analyzes the sub-question and decides which retrieval strategy—`vector_search`, `keyword_search`, or `hybrid_search`—is most appropriate. This adds a layer of dynamic decision-making that optimizes the retrieval process for each specific query.

In [19]:
class RetrievalDecision(BaseModel):
    strategy: Literal["vector_search", "keyword_search", "hybrid_search"]
    justification: str

retrieval_supervisor_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a retrieval strategy expert. Based on the user's query, you must decide the best retrieval strategy.\nYou have three options:\n1. `vector_search`: Best for conceptual, semantic, or similarity-based queries.\n2. `keyword_search`: Best for queries with specific, exact terms, names, or codes (e.g., 'Item 1A', 'Hopper architecture').\n3. `hybrid_search`: A good default that combines both, but may be less precise than a targeted strategy."""),
    ("human", "User Query: {sub_question}")
])

# Assuming reasoning_llm is already defined as AzureChatOpenAI
retrieval_supervisor_agent = retrieval_supervisor_prompt | reasoning_llm.with_structured_output(RetrievalDecision)
print("Retrieval Supervisor Agent created.")

# Test the supervisor
print("--- Testing Retrieval Supervisor Agent ---")
try:
    query1 = "revenue growth for the Compute & Networking segment in fiscal year 2023"
    decision1 = retrieval_supervisor_agent.invoke({"sub_question": query1})
    print(f"Query: '{query1}'")
    print(f"Decision: {decision1.strategy}, Justification: {decision1.justification}")

    query2 = "general sentiment about market competition and technological innovation"
    decision2 = retrieval_supervisor_agent.invoke({"sub_question": query2})
    print(f"Query: '{query2}'")
    print(f"Decision: {decision2.strategy}, Justification: {decision2.justification}")
except Exception as e:
    print(f"Error during retrieval supervisor agent test: {e}")


Retrieval Supervisor Agent created.
--- Testing Retrieval Supervisor Agent ---
Query: 'revenue growth for the Compute & Networking segment in fiscal year 2023'
Decision: keyword_search, Justification: The query contains specific terms such as 'revenue growth', 'Compute & Networking segment', and 'fiscal year 2023', which suggests that the user is looking for precise data or reports related to these exact terms.
Query: 'general sentiment about market competition and technological innovation'
Decision: vector_search, Justification: The query is focused on general sentiment, which is conceptual and relates to opinions and trends in market competition and technological innovation. A vector search is best suited for capturing the nuances and semantic relationships in such abstract topics.


#### 3.3.2. Implementing the Retrieval Strategies

Now we build our advanced retriever. We create a new vector store with our metadata-rich chunks. We then implement three distinct search functions: pure vector search, pure keyword search (BM25), and a hybrid approach that fuses the results using Reciprocal Rank Fusion (RRF). Our `retrieval_node` in the graph will use the decision from the `RetrievalSupervisor` to call the appropriate function.

In [20]:
import numpy as np

# Ensure required packages are installed
try:
    from rank_bm25 import BM25Okapi
except ImportError:
    print("Installing rank-bm25 package...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rank-bm25"])
    from rank_bm25 import BM25Okapi
    print("✓ rank-bm25 installed successfully")

try:
    from langchain_community.vectorstores import FAISS
except ImportError:
    print("Installing faiss-cpu...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faiss-cpu"])
    from langchain_community.vectorstores import FAISS
    print("✓ faiss-cpu installed successfully")

print("Creating advanced vector store with metadata...")

# Ensure embedding_function is available
if 'embedding_function' not in globals():
    raise ValueError("embedding_function not found. Please run the vector store creation cell first.")

# Ensure doc_chunks_with_metadata is available
if 'doc_chunks_with_metadata' not in globals():
    raise ValueError("doc_chunks_with_metadata not found. Please run the document processing cell first.")

# Use FAISS instead of ChromaDB
advanced_vector_store = FAISS.from_documents(
    documents=doc_chunks_with_metadata,
    embedding=embedding_function
)
print(f"Advanced vector store created with {len(doc_chunks_with_metadata)} documents.")

print("Building BM25 index for keyword search...")
tokenized_corpus = [doc.page_content.split(" ") for doc in doc_chunks_with_metadata]
doc_ids_list = [doc.metadata["id"] for doc in doc_chunks_with_metadata]
doc_map = {doc.metadata["id"]: doc for doc in doc_chunks_with_metadata}
bm25 = BM25Okapi(tokenized_corpus)

def vector_search_only(query: str, section_filter: str = None, k: int = 10):
    """Semantic search using FAISS with optional section filtering"""
    if section_filter and "Unknown" not in section_filter:
        # Filter documents by section before searching
        filtered_docs = [doc for doc in doc_chunks_with_metadata 
                        if doc.metadata.get("section", "") == section_filter]
        if filtered_docs:
            # Create temporary vector store for filtered docs
            temp_store = FAISS.from_documents(filtered_docs, embedding=embedding_function)
            return temp_store.similarity_search(query, k=k)
    
    return advanced_vector_store.similarity_search(query, k=k)

def bm25_search_only(query: str, k: int = 10):
    """Keyword-based search using BM25"""
    tokenized_query = query.split(" ")
    bm25_scores = bm25.get_scores(tokenized_query)
    top_k_indices = np.argsort(bm25_scores)[::-1][:k]
    return [doc_map[doc_ids_list[i]] for i in top_k_indices]

def hybrid_search(query: str, section_filter: str = None, k: int = 10):
    """Combines BM25 keyword search and semantic vector search using Reciprocal Rank Fusion"""
    # 1. Keyword Search (BM25)
    bm25_docs = bm25_search_only(query, k=k)

    # 2. Semantic Search (with metadata filtering)
    semantic_docs = vector_search_only(query, section_filter=section_filter, k=k)

    # 3. Reciprocal Rank Fusion (RRF)
    all_docs = {doc.metadata["id"]: doc for doc in bm25_docs + semantic_docs}.values()
    ranked_lists = [[doc.metadata["id"] for doc in bm25_docs], [doc.metadata["id"] for doc in semantic_docs]]

    rrf_scores = {}
    for doc_list in ranked_lists:
        for i, doc_id in enumerate(doc_list):
            if doc_id not in rrf_scores:
                rrf_scores[doc_id] = 0
            rrf_scores[doc_id] += 1 / (i + 61)  # RRF rank constant k = 60

    sorted_doc_ids = sorted(rrf_scores.keys(), key=lambda x: rrf_scores[x], reverse=True)
    final_docs = [doc_map[doc_id] for doc_id in sorted_doc_ids[:k]]
    return final_docs

print("All retrieval strategy functions ready.")

# Test Keyword Search with green hydrogen relevant query
print("\n--- Testing Keyword Search ---")
test_query = "green hydrogen cost benchmarks production India"
test_results = bm25_search_only(test_query, k=5)
print(f"Query: {test_query}")
print(f"Found {len(test_results)} documents.")
if test_results:
    print(f"Top result section: {test_results[0].metadata.get('section', 'Unknown')}")
    print(f"Top result preview: {test_results[0].page_content[:200]}...")

# Test Semantic Search
print("\n--- Testing Semantic Search ---")
test_query_semantic = "What are the key cost factors for green hydrogen production?"
semantic_results = vector_search_only(test_query_semantic, k=5)
print(f"Query: {test_query_semantic}")
print(f"Found {len(semantic_results)} documents.")
if semantic_results:
    print(f"Top result section: {semantic_results[0].metadata.get('section', 'Unknown')}")
    print(f"Top result preview: {semantic_results[0].page_content[:200]}...")

# Test Hybrid Search
print("\n--- Testing Hybrid Search ---")
test_query_hybrid = "policy frameworks incentives green hydrogen India"
hybrid_results = hybrid_search(test_query_hybrid, k=5)
print(f"Query: {test_query_hybrid}")
print(f"Found {len(hybrid_results)} documents.")
if hybrid_results:
    print(f"Top result section: {hybrid_results[0].metadata.get('section', 'Unknown')}")
    print(f"Top result preview: {hybrid_results[0].page_content[:200]}...")

Creating advanced vector store with metadata...
Advanced vector store created with 305 documents.
Building BM25 index for keyword search...
All retrieval strategy functions ready.

--- Testing Keyword Search ---
Query: green hydrogen cost benchmarks production India
Found 5 documents.
Top result section: India imported 90% of its panels from China before the implementation of safeguard
Top result preview: duties. To promote production in India, the Indian government levied a safeguard tax....

--- Testing Semantic Search ---
Query: What are the key cost factors for green hydrogen production?
Found 5 documents.
Top result section: India has embarked on the world’s biggest renewable capacity growth journey. The
Top result preview: government wants to boost the use of sustainable energy by investing heavily in green
energies. Energy security, electricity scarcity, energy access, among other things, and
climate change, played a r...

--- Testing Hybrid Search ---
Query: policy frameworks i

#### 3.3.3. Stage 2 (High Precision): Cross-Encoder Reranker.

After retrieving a broad set of `k` documents, we use a more computationally expensive but far more accurate **Cross-Encoder** model. Unlike embedding models (bi-encoders) that create vectors independently, a cross-encoder processes the query and each document *together*, yielding a much more nuanced relevance score. This allows us to re-rank the `k` candidates and select the top `n` with high confidence.

In [21]:
from sentence_transformers import CrossEncoder

print("Initializing CrossEncoder reranker...")
reranker = CrossEncoder(config["reranker_model"])

def rerank_documents_function(query: str, documents: List[Document]) -> List[Document]:
    if not documents: return []
    pairs = [(query, doc.page_content) for doc in documents]
    scores = reranker.predict(pairs)

    # Combine documents with their scores and sort
    doc_scores = list(zip(documents, scores))
    doc_scores.sort(key=lambda x: x[1], reverse=True)

    # Return top N documents
    reranked_docs = [doc for doc, score in doc_scores[:config["top_n_rerank"]]]
    return reranked_docs

print("Cross-Encoder ready.")

Initializing CrossEncoder reranker...
Cross-Encoder ready.


#### 3.3.4. Stage 3 (Contextual Distillation): Implementing logic to synthesize a concise context.

The final step in our retrieval funnel is to distill the top `n` highly relevant chunks into a single, clean paragraph of context. This removes redundancy and presents the information to the downstream agents in a clean, easy-to-process format. We create a dedicated 'Distiller Agent' for this.

In [22]:
distiller_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant. Your task is to synthesize the following retrieved document snippets into a single, concise paragraph.\nThe goal is to provide a clear and coherent context that directly answers the question: '{question}'.\nFocus on removing redundant information and organizing the content logically. Answer only with the synthesized context."""),
    ("human", "Retrieved Documents:\n{context}")
])

# Assuming reasoning_llm is already defined as AzureChatOpenAI
distiller_agent = distiller_prompt | reasoning_llm | StrOutputParser()
print("Contextual Distiller Agent created.")


Contextual Distiller Agent created.


### 3.4. Component 3: Tool Augmentation with Web Search

To answer questions about recent events or competitors, our agent needs to break out of its static knowledge base. We equip it with a web search tool using the Tavily Search API. The `planner_agent` will decide when to invoke this tool. The results from the web search will be formatted into LangChain `Document` objects, allowing them to be processed by the same reranking and compression pipeline as the documents retrieved from our vector store. This ensures a seamless integration of internal and external knowledge sources.

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

def web_search_function(query: str) -> List[Document]:
    results = web_search_tool.invoke({"query": query})
    return [Document(page_content=res["content"], metadata={"source": res["url"]}) for res in results]

print("Web search tool (Tavily) initialized.")

# Test the web search
print("--- Testing Web Search Tool ---")
test_query_web = "AMD AI chip strategy 2024"
test_results_web = web_search_function(test_query_web)
print(f"Found {len(test_results_web)} results for query: '{test_query_web}'")
if test_results_web:
    print(f"Top result snippet: {test_results_web[0].page_content[:250]}...")

/var/folders/w5/k1crj13j33n3_y1sj0nlz5yw0000gn/T/ipykernel_7519/126255760.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(k=3)


Web search tool (Tavily) initialized.
--- Testing Web Search Tool ---
Found 5 results for query: 'AMD AI chip strategy 2024'
Top result snippet: 2024 for its AI chips. [...] To meet the growing demand for AI chips, AMD has been actively working on expanding its production capacity. The company aims to increase its market share in the AI chip market, which is projected to reach USD 45 billion ...


### 3.5. Component 4: The Self-Critique and Control Flow Policy

#### 3.5.1. The "Update and Reflect" Step: An agent that synthesizes new findings into the `RAGState`'s reasoning history.

After each retrieval loop, the agent needs to integrate its new knowledge. The 'Reflection Agent' takes the distilled context from the current step and creates a concise summary. This summary is then appended to the `past_steps` list in our `RAGState`, forming a cumulative log of the agent's research journey.

In [24]:
reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a research assistant. Based on the retrieved context for the current sub-question, write a concise, one-sentence summary of the key findings.\nThis summary will be added to our research history. Be factual and to the point."""),
    ("human", "Current sub-question: {sub_question}\n\nDistilled context:\n{context}")
])
# Assuming reasoning_llm is already defined as AzureChatOpenAI
reflection_agent = reflection_prompt | reasoning_llm | StrOutputParser()
print("Reflection Agent created.")


Reflection Agent created.


#### 3.5.2. Policy Implementation (LLM-as-a-Judge): Prompting an LLM to inspect the current state and decide the next action.

This is the cognitive core of our agent's autonomy. The 'Policy Agent' acts as a supervisor. After each reflection step, it examines the *entire* research history (`past_steps`) in relation to the original question and the plan. It then makes a structured decision: `CONTINUE_PLAN` if more information is needed, or `FINISH` if the question has been comprehensively answered.

In [25]:
from pydantic import BaseModel
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
import json

class Decision(BaseModel):
    next_action: Literal["CONTINUE_PLAN", "FINISH"]
    justification: str

policy_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a master strategist specializing in energy sector research and analysis. Your role is to analyze the research progress and decide the next action.

You have the original question, the initial plan, and a log of completed steps with their summaries.

Your decision criteria:
- If the collected information in the Research History is sufficient to comprehensively answer the Original Question about green hydrogen, energy transition, policy frameworks, cost benchmarks, or related energy sector topics, decide to FINISH.
- Otherwise, if the plan is not yet complete or critical information is still missing, decide to CONTINUE_PLAN.

Consider whether you have:
- Sufficient data points, metrics, or benchmarks
- Complete policy framework information
- Adequate comparative analysis (if required)
- All necessary technical specifications or cost factors
- Comprehensive coverage of challenges and opportunities (if asked)"""),
    ("human", "Original Question: {question}\n\nInitial Plan:\n{plan}\n\nResearch History (Completed Steps):\n{history}")
])

# Ensure reasoning_llm is defined (should be from previous cell)
if 'reasoning_llm' not in locals():
    from langchain_openai import AzureChatOpenAI
    import os
    reasoning_llm = AzureChatOpenAI(
        azure_deployment=config["azure_deployment_name"],
        azure_endpoint=config["azure_endpoint"],
        api_version=config["azure_api_version"],
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
        temperature=0
    )

policy_agent = policy_prompt | reasoning_llm.with_structured_output(Decision)
print("Policy Agent created.")

# Test the policy agent with different states
print("--- Testing Policy Agent (Incomplete State) ---")
try:
    if 'test_plan' in locals() and test_plan:
        plan_str = json.dumps([s.model_dump() for s in test_plan.steps], indent=2)
        
        # Updated test history for green hydrogen document
        incomplete_history = """Step 1 Summary: The green hydrogen benchmarking document identifies key cost factors including electrolyzer costs, renewable energy prices, and infrastructure requirements. Initial cost benchmarks show green hydrogen production costs ranging from $3-7 per kg depending on renewable energy source and scale."""
        
        # Use the complex_query_adv if available, otherwise use a default query
        query_to_use = complex_query_adv if 'complex_query_adv' in globals() else "What are the key cost benchmarks for green hydrogen production in India?"
        
        decision1 = policy_agent.invoke({
            "question": query_to_use, 
            "plan": plan_str, 
            "history": incomplete_history
        })
        print(f"Decision: {decision1.next_action}")
        print(f"Justification: {decision1.justification}")

        # Complete history with more comprehensive information
        complete_history = incomplete_history + """
Step 2 Summary: Policy frameworks in India include the National Green Hydrogen Mission with targets of 5 million tonnes annual production by 2030, production-linked incentives, and regulatory support for electrolyzer manufacturing.
Step 3 Summary: Comparative analysis shows India's green hydrogen costs are competitive with international benchmarks, with potential for further reduction through scale, technology improvements, and renewable energy cost reductions.
Step 4 Summary: Key challenges include high initial capital costs, intermittent renewable energy supply, and infrastructure development needs. Opportunities include abundant renewable resources, government support, and growing market demand."""
        
        decision2 = policy_agent.invoke({
            "question": query_to_use, 
            "plan": plan_str, 
            "history": complete_history
        })
        print("\n--- Testing Policy Agent (Complete State) ---")
        print(f"Decision: {decision2.next_action}")
        print(f"Justification: {decision2.justification}")
    else:
        print("test_plan not defined. Running sample test with mock data...")
        
        # Sample test without test_plan
        sample_plan = json.dumps([
            {"sub_question": "What are the cost benchmarks for green hydrogen?", "tool": "search_documents"},
            {"sub_question": "What policy frameworks support green hydrogen in India?", "tool": "search_documents"},
            {"sub_question": "How do costs compare to conventional hydrogen?", "tool": "search_documents"}
        ], indent=2)
        
        incomplete_history = "Step 1 Summary: Cost benchmarks identified for green hydrogen production."
        complete_history = incomplete_history + "\nStep 2 Summary: Policy frameworks documented.\nStep 3 Summary: Cost comparison completed."
        
        sample_query = "What are the key cost benchmarks and policy frameworks for green hydrogen in India?"
        
        decision1 = policy_agent.invoke({
            "question": sample_query,
            "plan": sample_plan,
            "history": incomplete_history
        })
        print(f"Decision (Incomplete): {decision1.next_action}")
        print(f"Justification: {decision1.justification}")
        
        decision2 = policy_agent.invoke({
            "question": sample_query,
            "plan": sample_plan,
            "history": complete_history
        })
        print(f"\nDecision (Complete): {decision2.next_action}")
        print(f"Justification: {decision2.justification}")
        
except Exception as e:
    print(f"Error during policy agent test: {e}")
    import traceback
    traceback.print_exc()

Policy Agent created.
--- Testing Policy Agent (Incomplete State) ---
Decision: CONTINUE_PLAN
Justification: While initial cost benchmarks for green hydrogen production in India have been identified, further information is needed on performance metrics, a comparative analysis with conventional hydrogen production methods, and the main factors affecting cost competitiveness to comprehensively answer the original question.

--- Testing Policy Agent (Complete State) ---
Decision: FINISH
Justification: The research history provides comprehensive information on key cost benchmarks, performance metrics, comparative analysis with conventional hydrogen production, and the main factors affecting cost competitiveness for green hydrogen production in India. All necessary aspects of the original question have been addressed.


#### 3.5.3. Defining Robust Stopping Criteria

Our system needs clear and robust conditions to stop the reasoning loop. We have three such criteria:
1.  **Policy Decision:** The primary stopping condition is when the `policy_agent` confidently decides to `FINISH`.
2.  **Plan Completion:** If the agent has executed every step in its plan, it will naturally conclude its work.
3.  **Max Iterations:** As a safeguard against infinite loops or runaway processes, we enforce a hard limit (`max_reasoning_iterations` from our config) on the number of research cycles.

## Part 4: Assembly with LangGraph - Orchestrating the Reasoning Loop

### 4.1. Code Dependency: Defining the Graph Nodes

Now, we translate our conceptual components into concrete graph nodes. Each node is a Python function that accepts the `RAGState` dictionary, performs its designated task, and returns a dictionary containing the state updates. We add a new `web_search_node` to handle the external search tool, and we modify the `retrieval_node` to incorporate the adaptive strategy chosen by our new Supervisor agent.

In [26]:
def get_past_context_str(past_steps: List[PastStep]) -> str:
    return "\n\n".join([f"Step {s['step_index']}: {s['sub_question']}\nSummary: {s['summary']}" for s in past_steps])

def plan_node(state: RAGState) -> Dict:
    console.print("--- 🧠: Generating Plan ---")
    plan = planner_agent.invoke({"question": state["original_question"]})
    rprint(plan)
    return {"plan": plan, "current_step_index": 0, "past_steps": []}

def retrieval_node(state: RAGState) -> Dict:
    current_step_index = state["current_step_index"]
    current_step = state["plan"].steps[current_step_index]
    console.print(f"--- 🔍: Retrieving from 10-K (Step {current_step_index + 1}: {current_step.sub_question}) ---")
    past_context = get_past_context_str(state['past_steps'])
    rewritten_query = query_rewriter_agent.invoke({
        "sub_question": current_step.sub_question,
        "keywords": current_step.keywords,
        "past_context": past_context
    })
    console.print(f"  Rewritten Query: {rewritten_query}")

    # NEW: Adaptive Retrieval Strategy
    retrieval_decision = retrieval_supervisor_agent.invoke({"sub_question": rewritten_query})
    console.print(f"  Supervisor Decision: Use `{retrieval_decision.strategy}`. Justification: {retrieval_decision.justification}")

    if retrieval_decision.strategy == 'vector_search':
        retrieved_docs = vector_search_only(rewritten_query, section_filter=current_step.document_section, k=config['top_k_retrieval'])
    elif retrieval_decision.strategy == 'keyword_search':
        retrieved_docs = bm25_search_only(rewritten_query, k=config['top_k_retrieval'])
    else: # hybrid_search
        retrieved_docs = hybrid_search(rewritten_query, section_filter=current_step.document_section, k=config['top_k_retrieval'])

    return {"retrieved_docs": retrieved_docs}

def web_search_node(state: RAGState) -> Dict:
    current_step_index = state["current_step_index"]
    current_step = state["plan"].steps[current_step_index]
    console.print(f"--- 🌐: Searching Web (Step {current_step_index + 1}: {current_step.sub_question}) ---")
    past_context = get_past_context_str(state['past_steps'])
    rewritten_query = query_rewriter_agent.invoke({
        "sub_question": current_step.sub_question,
        "keywords": current_step.keywords,
        "past_context": past_context
    })
    console.print(f"  Rewritten Query: {rewritten_query}")
    retrieved_docs = web_search_function(rewritten_query)
    return {"retrieved_docs": retrieved_docs}

def rerank_node(state: RAGState) -> Dict:
    console.print("--- 🎯: Reranking Documents ---")
    current_step_index = state["current_step_index"]
    current_step = state["plan"].steps[current_step_index]
    reranked_docs = rerank_documents_function(current_step.sub_question, state["retrieved_docs"])
    console.print(f"  Reranked to top {len(reranked_docs)} documents.")
    return {"reranked_docs": reranked_docs}

def compression_node(state: RAGState) -> Dict:
    console.print("--- ✂️: Distilling Context ---")
    current_step_index = state["current_step_index"]
    current_step = state["plan"].steps[current_step_index]
    context = format_docs(state["reranked_docs"])
    synthesized_context = distiller_agent.invoke({"question": current_step.sub_question, "context": context})
    console.print(f"  Distilled Context Snippet: {synthesized_context[:200]}...")
    return {"synthesized_context": synthesized_context}

def reflection_node(state: RAGState) -> Dict:
    console.print("--- 🤔: Reflecting on Findings ---")
    current_step_index = state["current_step_index"]
    current_step = state["plan"].steps[current_step_index]
    summary = reflection_agent.invoke({"sub_question": current_step.sub_question, "context": state['synthesized_context']})
    console.print(f"  Summary: {summary}")
    new_past_step = {
        "step_index": current_step_index + 1,
        "sub_question": current_step.sub_question,
        "retrieved_docs": state['reranked_docs'],
        "summary": summary
    }
    return {"past_steps": state["past_steps"] + [new_past_step], "current_step_index": current_step_index + 1}

def final_answer_node(state: RAGState) -> Dict:
    console.print("--- ✅: Generating Final Answer with Citations ---")
    # Create a consolidated context with metadata for citation
    final_context = ""
    for i, step in enumerate(state['past_steps']):
        final_context += f"\n--- Findings from Research Step {i+1} ---\n"
        for doc in step['retrieved_docs']:
            source = doc.metadata.get('section') or doc.metadata.get('source')
            final_context += f"Source: {source}\nContent: {doc.page_content}\n\n"

    final_answer_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert financial analyst. Synthesize the research findings from internal documents and web searches into a comprehensive, multi-paragraph answer for the user's original question.\nYour answer must be grounded in the provided context. At the end of any sentence that relies on specific information, you MUST add a citation. For 10-K documents, use [Source: <section title>]. For web results, use [Source: <URL>]."""),
        ("human", "Original Question: {question}\n\nResearch History and Context:\n{context}")
    ])

    # Correct way to invoke the LLM with a prompt template and ensure string output
    final_answer_chain = final_answer_prompt | reasoning_llm | StrOutputParser() # Added StrOutputParser
    final_answer = final_answer_chain.invoke({"question": state['original_question'], "context": final_context})
    return {"final_answer": final_answer}

print("All graph nodes defined successfully.")

All graph nodes defined successfully.


### 4.2. Code Dependency: Defining the Conditional Edges - Implementing the Self-Critique Policy Logic

We now define the logic that controls the flow of our graph. We add a `route_by_tool` function that checks the plan and directs the agent to either the `retrieval_node` or the `web_search_node`. The `should_continue_node` remains the primary controller for the main reasoning loop, implementing our stopping criteria.

In [27]:
# Update route_by_tool to map old tool names to new ones
def route_by_tool(state: RAGState) -> str:
    current_step_index = state.get("current_step_index", 0)
    if current_step_index >= len(state["plan"].steps):
        return "search_web"  # Default if out of bounds
    
    current_step = state["plan"].steps[current_step_index]
    tool = current_step.tool
    
    # Map old tool names to new ones for compatibility
    if tool == "search_10k":
        return "search_documents"
    elif tool == "search_documents":
        return "search_documents"
    elif tool == "search_web":
        return "search_web"
    else:
        # Default to document search for unknown tools
        return "search_documents"

# Ensure the graph uses the updated routing
# Rebuild the graph with correct edges if needed
from langgraph.graph import StateGraph, END

# If graph is already defined, we just need to ensure route_by_tool is updated
# The graph edges should already be correct from the previous cell

print("route_by_tool function updated to map 'search_10k' -> 'search_documents'")

def should_continue_node(state: RAGState) -> str:
    console.print("--- 🚦: Evaluating Policy ---")
    current_step_index = state["current_step_index"]

    if current_step_index >= len(state["plan"].steps):
        console.print("  -> Plan complete. Finishing.")
        return "finish"

    if current_step_index >= config["max_reasoning_iterations"]:
        console.print("  -> Max iterations reached. Finishing.")
        return "finish"

    # Check if the last retrieval step failed to find documents
    if not state["reranked_docs"]:
        console.print("  -> Retrieval failed for the last step. Continuing with next step in plan.")
        return "continue"

    history = get_past_context_str(state['past_steps'])
    plan_str = json.dumps([s.model_dump() for s in state['plan'].steps]) # Changed .dict() to .model_dump()
    decision = policy_agent.invoke({"question": state["original_question"], "plan": plan_str, "history": history})
    console.print(f"  -> Decision: {decision.next_action} | Justification: {decision.justification}")

    if decision.next_action == "FINISH":
        return "finish"
    else: # CONTINUE_PLAN
        return "continue"

print("Conditional edge logic functions defined.")

route_by_tool function updated to map 'search_10k' -> 'search_documents'
Conditional edge logic functions defined.


### 4.3. Building the `StateGraph`: Wiring the Deep Thinking RAG Machine

Now we instantiate the `StateGraph` and assemble our more advanced cognitive architecture. The key change is adding a conditional entry point after the `plan` node. This `route_by_tool` edge will direct the agent to the correct tool for the current step. After each tool execution and subsequent processing, the graph flows to the `reflect` node, which then loops back to the tool router for the next step in the plan.

In [28]:
# Ensure langgraph is installed
try:
    from langgraph.graph import StateGraph, END
except ImportError:
    print("Installing langgraph package...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "langgraph"])
    from langgraph.graph import StateGraph, END
    print("✓ langgraph installed successfully")

# Ensure RAGState is defined (should be from previous cells)
if 'RAGState' not in globals():
    from typing import TypedDict, List, Annotated
    from langchain_core.documents import Document
    
    class RAGState(TypedDict):
        question: str
        plan: str
        retrieved_docs: List[Document]
        web_results: List[Document]
        reranked_docs: List[Document]
        compressed_context: str
        research_history: str
        final_answer: str
        current_step: int
        max_steps: int

# Ensure all node functions are defined
# These should be defined in previous cells, but we'll check
required_nodes = ['plan_node', 'retrieval_node', 'web_search_node', 'rerank_node', 
                 'compression_node', 'reflection_node', 'final_answer_node', 
                 'route_by_tool', 'should_continue_node']

missing_nodes = [node for node in required_nodes if node not in globals()]
if missing_nodes:
    print(f"⚠ Warning: The following node functions are not defined: {missing_nodes}")
    print("Please ensure all node functions are defined before building the graph.")

from langgraph.graph import StateGraph, END

graph = StateGraph(RAGState)

# Add nodes
graph.add_node("plan", plan_node)
graph.add_node("retrieve_documents", retrieval_node)  # Changed from "retrieve_10k" to "retrieve_documents"
graph.add_node("retrieve_web", web_search_node)
graph.add_node("rerank", rerank_node)
graph.add_node("compress", compression_node)
graph.add_node("reflect", reflection_node)
graph.add_node("generate_final_answer", final_answer_node)
# Routing node for dynamic tool selection
graph.add_node("choose_next_tool", lambda state: state)

graph.set_entry_point("plan")

# After initial plan, direct to the routing point
graph.add_edge("plan", "choose_next_tool")

# Use 'choose_next_tool' as the source for dynamic routing to the correct tool
graph.add_conditional_edges(
    "choose_next_tool",  # Source is our routing node
    route_by_tool,  # This function will decide the next actual tool node
    {
        "search_documents": "retrieve_documents",  # Changed from "search_10k" to "search_documents"
        "search_web": "retrieve_web",
    },
)

# Common pipeline after retrieval
graph.add_edge("retrieve_documents", "rerank")  # Changed from "retrieve_10k"
graph.add_edge("retrieve_web", "rerank")
graph.add_edge("rerank", "compress")
graph.add_edge("compress", "reflect")

# After reflection, use should_continue_node to decide if we keep looping or finish
graph.add_conditional_edges(
    "reflect",  # Source is 'reflect'
    should_continue_node,
    {
        "continue": "choose_next_tool",  # If continue, go back to the routing point for the next step
        "finish": "generate_final_answer",
    },
)

graph.add_edge("generate_final_answer", END)

print("StateGraph constructed successfully.")
print("Graph nodes:", list(graph.nodes.keys()))

StateGraph constructed successfully.
Graph nodes: ['plan', 'retrieve_documents', 'retrieve_web', 'rerank', 'compress', 'reflect', 'generate_final_answer', 'choose_next_tool']


### 4.4. Compiling and Visualizing the Iterative Workflow

The final step is to compile our graph definition into an executable `Runnable`. We then generate a visual diagram of the graph. The new diagram will clearly show the branching logic where the agent decides between its internal knowledge base (`retrieve_10k`) and its external web search tool (`retrieve_web`).

In [29]:
deep_thinking_rag_graph = graph.compile()
print("Graph compiled successfully.")

try:
    from IPython.display import Image, display
    # Correctly call get_graph() before draw_png()
    png_image = deep_thinking_rag_graph.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}. Please ensure pygraphviz is installed.")

Graph compiled successfully.
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed.


## Part 5: Redemption - Running the Deep Thinking Pipeline on Our Challenge Query

### 5.1. Invoking the Graph: A Step-by-Step Trace of the Full Reasoning Process

With our graph compiled, we can now invoke it with our complex, multi-source query. We use the `.stream()` method to observe the agent's execution in real-time. The trace will now demonstrate the agent's ability to first query its internal knowledge base, and then seamlessly switch to its web search tool to gather the external information required to fully answer the user's question.

In [30]:
# Recompile the graph to use the updated route_by_tool function
# If the graph was already compiled, we need to rebuild it

from langgraph.graph import StateGraph, END

# Check if graph needs to be rebuilt
if 'deep_thinking_rag_graph' in globals():
    print("Graph already exists. Verifying edges...")
    # The graph should use the current route_by_tool function
    # If it was compiled before, we may need to rebuild
    print("Note: If you updated route_by_tool, you may need to rebuild the graph")
else:
    print("Graph not found. Please build it first.")

# Prepare the graph input
final_state = None

graph_input = {
    "original_question": complex_query_adv,
    "question": complex_query_adv,
    "plan": None,
    "retrieved_docs": [],
    "web_results": [],
    "reranked_docs": [],
    "compressed_context": "",
    "research_history": "",
    "final_answer": "",
    "current_step": 0,
    "current_step_index": 0,
    "past_steps": [],
    "max_steps": config.get("max_reasoning_iterations", 7)
}

print("\n--- Invoking Deep Thinking RAG Graph ---")
print(f"Query: {complex_query_adv[:100]}...")

# Increase recursion_limit to allow more graph steps
try:
    for chunk in deep_thinking_rag_graph.stream(
        graph_input, 
        stream_config={"recursion_limit": 50}, 
        stream_mode="values"
    ):
        final_state = chunk
        # Print progress
        if "current_step_index" in chunk:
            step_idx = chunk.get("current_step_index", 0)
            print(f"✓ Step {step_idx} completed")
            if "plan" in chunk and chunk["plan"]:
                plan = chunk["plan"]
                if hasattr(plan, 'steps') and step_idx < len(plan.steps):
                    step = plan.steps[step_idx]
                    print(f"  Tool: {step.tool}, Question: {step.sub_question[:60]}...")
    
    print("\n--- Graph Stream Finished ---")
    
    if final_state:
        if "final_answer" in final_state and final_state["final_answer"]:
            print("\n" + "=" * 60)
            print("FINAL ANSWER")
            print("=" * 60)
            print(final_state["final_answer"])
            print("=" * 60)
        else:
            print("\n⚠ No final answer generated")
            if "research_history" in final_state:
                print("\nResearch History:")
                print(final_state["research_history"][:500] + "...")
        
except KeyError as e:
    print(f"\n❌ KeyError: {e}")
    print("\nThis usually means:")
    print("1. route_by_tool is returning a tool name that doesn't match graph edges")
    print("2. The graph edges are: 'search_documents' and 'search_web'")
    print("3. But route_by_tool is returning: 'search_10k'")
    print("\nSolution: Ensure route_by_tool function is defined above and maps 'search_10k' -> 'search_documents'")
    raise
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
    raise

Graph already exists. Verifying edges...
Note: If you updated route_by_tool, you may need to rebuild the graph

--- Invoking Deep Thinking RAG Graph ---
Query: What are the key cost benchmarks and performance metrics for green hydrogen production in India as o...
✓ Step 0 completed


--- 🧠: Generating Plan ---

Plan(
│   steps=[
│   │   Step(
│   │   │   sub_question='What are the key cost benchmarks for green hydrogen production in India?',
│   │   │   justification='Understanding the cost benchmarks is essential to evaluate the economic viability of green hydrogen production in India.',
│   │   │   tool='search_documents',
│   │   │   keywords=['cost benchmarks', 'green hydrogen production', 'India'],
│   │   │   document_section='cost benchmarks'
│   │   ),
│   │   Step(
│   │   │   sub_question='What are the performance metrics for green hydrogen production in India?',
│   │   │   justification='Performance metrics will provide insights into the efficiency and effectiveness of green hydrogen production processes.',
│   │   │   tool='search_documents',
│   │   │   keywords=['performance metrics', 'green hydrogen production', 'India'],
│   │   │   document_section='performance metrics'
│   │   ),
│   │   Step(
│   │   │   sub_question='How do the cost benchmarks and performance metrics of green hydrogen compare to conventional hydrogen production methods?',
│   │   │   justification='A comparative analysis will highlight the advantages and disadvantages of green hydrogen relative to conventional methods, which is crucial for understanding its market position.',
│   │   │   tool='search_documents',
│   │   │   keywords=['comparison', 'green hydrogen', 'conventional hydrogen', 'cost', 'performance'],
│   │   │   document_section='comparative analysis'
│   │   ),
│   │   Step(
│   │   │   sub_question='What are the main factors affecting the cost competitiveness of green hydrogen production in India?',
│   │   │   justification='Identifying these factors will help in understanding the challenges and opportunities for green hydrogen in the Indian market.',
│   │   │   tool='search_documents',
│   │   │   keywords=['cost competitiveness', 'factors', 'green hydrogen', 'India'],
│   │   │   document_section='challenges and opportunities'
│   │   )
│   ]
)

✓ Step 0 completed
  Tool: search_documents, Question: What are the key cost benchmarks for green hydrogen producti...
✓ Step 0 completed
  Tool: search_documents, Question: What are the key cost benchmarks for green hydrogen producti...


--- 🔍: Retrieving from 10-K (Step 1: What are the key cost benchmarks for green hydrogen production in India?) ---

...

--- 🚦: Evaluating Policy ---

-> Plan complete. Finishing.

✓ Step 4 completed


--- ✅: Generating Final Answer with Citations ---

✓ Step 4 completed

--- Graph Stream Finished ---

FINAL ANSWER
The production of green hydrogen in India is increasingly being recognized as a pivotal component of the country's energy transition strategy. Key cost benchmarks and performance metrics for green hydrogen production are influenced by several factors, including the cost of renewable energy sources, technology adoption, and government policies. As outlined in various benchmarking documents, the cost of producing green hydrogen is primarily driven by the price of electricity, which is expected to decrease as renewable energy capacity expands. Currently, the cost of green hydrogen production in India is estimated to be around ₹300-400 per kg, with projections suggesting that this could drop to ₹150-200 per kg by 2030 as renewable energy becomes more prevalent and efficient [Source: Energies 2023, 16, 5491].

In comparison, conventional hydrogen production methods, such as steam methane reforming (SMR), typically have lower up

In [31]:
import numpy as np

# Ensure required packages are installed
try:
    from rank_bm25 import BM25Okapi
except ImportError:
    print("Installing rank-bm25 package...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rank-bm25"])
    from rank_bm25 import BM25Okapi
    print("✓ rank-bm25 installed successfully")

try:
    from langchain_community.vectorstores import FAISS
except ImportError:
    print("Installing faiss-cpu...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faiss-cpu"])
    from langchain_community.vectorstores import FAISS
    print("✓ faiss-cpu installed successfully")

print("Creating advanced vector store with metadata...")

# Ensure embedding_function is available
if 'embedding_function' not in globals():
    raise ValueError("embedding_function not found. Please run the vector store creation cell first.")

# Ensure doc_chunks_with_metadata is available
if 'doc_chunks_with_metadata' not in globals():
    raise ValueError("doc_chunks_with_metadata not found. Please run the document processing cell first.")

# Use FAISS instead of ChromaDB
advanced_vector_store = FAISS.from_documents(
    documents=doc_chunks_with_metadata,
    embedding=embedding_function
)
print(f"Advanced vector store created with {len(doc_chunks_with_metadata)} documents.")

print("Building BM25 index for keyword search...")
tokenized_corpus = [doc.page_content.split(" ") for doc in doc_chunks_with_metadata]
doc_ids_list = [doc.metadata["id"] for doc in doc_chunks_with_metadata]
doc_map = {doc.metadata["id"]: doc for doc in doc_chunks_with_metadata}
bm25 = BM25Okapi(tokenized_corpus)

def vector_search_only(query: str, section_filter: str = None, k: int = 10):
    """Semantic search using FAISS with optional section filtering"""
    if section_filter and "Unknown" not in section_filter:
        # Filter documents by section before searching
        filtered_docs = [doc for doc in doc_chunks_with_metadata 
                        if doc.metadata.get("section", "") == section_filter]
        if filtered_docs:
            # Create temporary vector store for filtered docs
            temp_store = FAISS.from_documents(filtered_docs, embedding=embedding_function)
            return temp_store.similarity_search(query, k=k)
    
    return advanced_vector_store.similarity_search(query, k=k)

def bm25_search_only(query: str, k: int = 10):
    """Keyword-based search using BM25"""
    tokenized_query = query.split(" ")
    bm25_scores = bm25.get_scores(tokenized_query)
    top_k_indices = np.argsort(bm25_scores)[::-1][:k]
    return [doc_map[doc_ids_list[i]] for i in top_k_indices]

def hybrid_search(query: str, section_filter: str = None, k: int = 10):
    """Combines BM25 keyword search and semantic vector search using Reciprocal Rank Fusion"""
    # 1. Keyword Search (BM25)
    bm25_docs = bm25_search_only(query, k=k)

    # 2. Semantic Search (with metadata filtering)
    semantic_docs = vector_search_only(query, section_filter=section_filter, k=k)

    # 3. Reciprocal Rank Fusion (RRF)
    all_docs = {doc.metadata["id"]: doc for doc in bm25_docs + semantic_docs}.values()
    ranked_lists = [[doc.metadata["id"] for doc in bm25_docs], [doc.metadata["id"] for doc in semantic_docs]]

    rrf_scores = {}
    for doc_list in ranked_lists:
        for i, doc_id in enumerate(doc_list):
            if doc_id not in rrf_scores:
                rrf_scores[doc_id] = 0
            rrf_scores[doc_id] += 1 / (i + 61)  # RRF rank constant k = 60

    sorted_doc_ids = sorted(rrf_scores.keys(), key=lambda x: rrf_scores[x], reverse=True)
    final_docs = [doc_map[doc_id] for doc_id in sorted_doc_ids[:k]]
    return final_docs

print("All retrieval strategy functions ready.")

# Test Keyword Search with green hydrogen relevant query
print("\n--- Testing Keyword Search ---")
test_query = "green hydrogen cost benchmarks production India"
test_results = bm25_search_only(test_query, k=5)
print(f"Query: {test_query}")
print(f"Found {len(test_results)} documents.")
if test_results:
    print(f"Top result section: {test_results[0].metadata.get('section', 'Unknown')}")
    print(f"Top result preview: {test_results[0].page_content[:200]}...")

# Test Semantic Search
print("\n--- Testing Semantic Search ---")
test_query_semantic = "What are the key cost factors for green hydrogen production?"
semantic_results = vector_search_only(test_query_semantic, k=5)
print(f"Query: {test_query_semantic}")
print(f"Found {len(semantic_results)} documents.")
if semantic_results:
    print(f"Top result section: {semantic_results[0].metadata.get('section', 'Unknown')}")
    print(f"Top result preview: {semantic_results[0].page_content[:200]}...")

# Test Hybrid Search
print("\n--- Testing Hybrid Search ---")
test_query_hybrid = "policy frameworks incentives green hydrogen India"
hybrid_results = hybrid_search(test_query_hybrid, k=5)
print(f"Query: {test_query_hybrid}")
print(f"Found {len(hybrid_results)} documents.")
if hybrid_results:
    print(f"Top result section: {hybrid_results[0].metadata.get('section', 'Unknown')}")
    print(f"Top result preview: {hybrid_results[0].page_content[:200]}...")

Creating advanced vector store with metadata...
Advanced vector store created with 305 documents.
Building BM25 index for keyword search...
All retrieval strategy functions ready.

--- Testing Keyword Search ---
Query: green hydrogen cost benchmarks production India
Found 5 documents.
Top result section: India imported 90% of its panels from China before the implementation of safeguard
Top result preview: duties. To promote production in India, the Indian government levied a safeguard tax....

--- Testing Semantic Search ---
Query: What are the key cost factors for green hydrogen production?
Found 5 documents.
Top result section: India has embarked on the world’s biggest renewable capacity growth journey. The
Top result preview: government wants to boost the use of sustainable energy by investing heavily in green
energies. Energy security, electricity scarcity, energy access, among other things, and
climate change, played a r...

--- Testing Hybrid Search ---
Query: policy frameworks i

### 5.2. Analyzing the Final High-Quality Output with Full Provenance

The agent has successfully executed its plan, using the right tool for each step. Now, we examine the `final_answer` stored in the terminal state. Unlike the baseline's failure, we expect a cohesive, multi-part answer that successfully synthesizes information from two different sources into a single analytical response, complete with citations to both the 10-K and the web.

In [32]:
console.print("--- DEEP THINKING RAG FINAL ANSWER ---")
console.print(Markdown(final_state['final_answer']))

--- DEEP THINKING RAG FINAL ANSWER ---

The production of green hydrogen in India is increasingly being recognized as a pivotal component of the country's 
energy transition strategy. Key cost benchmarks and performance metrics for green hydrogen production are          
influenced by several factors, including the cost of renewable energy sources, technology adoption, and government 
policies. As outlined in various benchmarking documents, the cost of producing green hydrogen is primarily driven  
by the price of electricity, which is expected to decrease as renewable energy capacity expands. Currently, the    
cost of green hydrogen production in India is estimated to be around ₹300-400 per kg, with projections suggesting  
that this could drop to ₹150-200 per kg by 2030 as renewable energy becomes more prevalent and efficient [Source:  
Energies 2023, 16, 5491].                                                                                          

In comparison, conventional hydrogen production methods, such as steam methane reforming (SMR), typically have     
lower upfront costs but are heavily reliant on fossil fuels, making them less sustainable in the long term. The    
cost of hydrogen produced via SMR is approximately ₹150-200 per kg, but this does not account for the environmental
costs associated with carbon emissions [Source: Energies 2023, 16, 5491]. The stark difference in sustainability   
and long-term viability between green hydrogen and conventional methods highlights the importance of transitioning 
to cleaner energy sources.                                                                                         

Several factors affect the cost competitiveness of green hydrogen production in India. Firstly, the availability   
and cost of renewable energy sources, particularly solar and wind, play a crucial role. India has significant      
potential for solar energy, with estimates suggesting a capacity of around 748.99 GW, which can significantly lower
the cost of green hydrogen production as solar energy becomes more widely utilized [Source: MNRE]. Secondly,       
technological advancements in electrolysis and hydrogen storage are essential for improving efficiency and reducing
costs. The current efficiency of electrolysis systems is around 60-70%, and ongoing research aims to enhance this  
to 80% or more, which would further decrease production costs [Source: Energies 2023, 16, 5491].                   

Government policies and incentives also significantly impact the competitiveness of green hydrogen. Initiatives    
such as the National Hydrogen Mission aim to promote the development of hydrogen technologies and infrastructure,  
which can help reduce costs through economies of scale and increased investment in research and development        
[Source: Energies 2023, 16, 5491]. Additionally, the implementation of carbon pricing could make conventional      
hydrogen production methods less attractive, further enhancing the competitiveness of green hydrogen.              

In summary, while green hydrogen production in India currently faces higher costs compared to conventional methods,
the potential for cost reduction through advancements in renewable energy, technology, and supportive government   
policies presents a promising outlook. As these factors converge, green hydrogen could become a cornerstone of     
India's energy strategy, aligning with global sustainability goals and reducing reliance on fossil fuels.

### 5.3. Side-by-Side Comparison: Vanilla RAG vs. Deep Thinking RAG

| Feature                 | Vanilla RAG (Failed)                                                                                                                              | Deep Thinking RAG (Success)                                                                                                                                                                                                                                                                                            |
|-------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Cognitive Model**     | Linear, stateless, one-shot retrieval.                                                                                                            | Cyclical, stateful, multi-step reasoning loop.                                                                                                                                                                                                                                                                         |
| **Planning**            | None. The entire complex query is treated as a single search.                                                                                     | Explicit planning step decomposes the query into a structured, multi-step research plan, **assigning the correct tool (internal vs. web) to each step.**                                                                                                                                                                 |
| **Retrieval Strategy**  | Naive semantic search on a single static source.                                                                             | **Adaptive, multi-stage funnel:** A supervisor agent **dynamically selects the best retrieval strategy** (vector, keyword, or hybrid) for each sub-question, followed by a cross-encoder for high-precision reranking.                                                                                                         |
| **Knowledge Source**    | Restricted to the single, static 10-K document.                                                                                                   | **Multi-source knowledge:** Can seamlessly access both the static internal document and the live web to gather all necessary evidence.                                                                                                                                                                                           |
| **Answer Quality**      | Completely failed to answer the second part of the query due to a lack of information. Unable to perform any synthesis.                                     | Answered all parts of the query comprehensively. **Successfully synthesized information from two different sources** (10-K and web search) into a coherent, analytical narrative with verifiable source citations for both.                                                                                                    |

## Part 6: A Production-Grade Evaluation Framework

To move from anecdotal success to objective validation, we employ a rigorous, automated evaluation framework. We will use the **RAGAs** (RAG Assessment) library to score both our baseline and advanced pipelines across a suite of metrics designed to quantify the quality and reliability of RAG systems.

### 6.1. Evaluation Metrics Overview
**Context Precision & Recall** measure the quality of the retrieved information. Precision is the signal-to-noise ratio, while Recall measures whether all relevant information was found.

**Answer Faithfulness** measures whether the answer is grounded in the provided context, preventing hallucination.

**Answer Correctness** measures how well the answer addresses the user's query when compared to a 'ground truth' ideal answer.

### 6.2. Code Dependency: Implementing an Automated Evaluation with RAGAs

We construct a `Dataset` object for evaluation. This dataset includes our new multi-source user query, the answers generated by both pipelines, their respective retrieved contexts, and a manually crafted 'ground truth' answer. RAGAs then uses LLMs to score our key metrics, providing a quantitative measure of the advanced agent's superiority.

In [33]:
# Enhanced manual evaluation with comprehensive metrics including RAGAS-style metrics
print("=" * 80)
print("COMPREHENSIVE EVALUATION - Baseline vs Deep Thinking RAG")
print("=" * 80)

import pandas as pd
import re
from collections import Counter

def comprehensive_evaluation(question, answer, ground_truth, contexts, model_name="Model"):
    """Comprehensive evaluation metrics including RAGAS-style metrics."""
    metrics = {}
    
    # Basic metrics
    metrics['answer_length'] = len(answer)
    metrics['word_count'] = len(answer.split())
    metrics['sentence_count'] = len(re.split(r'[.!?]+', answer))
    metrics['avg_words_per_sentence'] = metrics['word_count'] / max(metrics['sentence_count'], 1)
    
    # Key terms coverage (energy sector specific)
    energy_keywords = [
        'green hydrogen', 'hydrogen', 'renewable', 'energy', 'transition',
        'cost', 'benchmark', 'production', 'India', 'policy', 'framework',
        'electrolyzer', 'LCOH', 'levelized', 'incentive', 'target', '2030',
        'renewable energy', 'solar', 'wind', 'infrastructure', 'challenge',
        'opportunity', 'market', 'investment', 'technology', 'efficiency'
    ]
    
    answer_lower = answer.lower()
    found_keywords = [kw for kw in energy_keywords if kw in answer_lower]
    metrics['key_terms_found'] = len(found_keywords)
    metrics['key_terms_coverage'] = len(found_keywords) / len(energy_keywords)
    metrics['key_terms_list'] = found_keywords
    
    # Technical terms (numbers, percentages, specific values)
    numbers = re.findall(r'\$?\d+[.,]?\d*[%]?', answer)
    metrics['numerical_data_points'] = len(numbers)
    metrics['has_specific_values'] = 1 if len(numbers) > 0 else 0
    
    # Context usage
    if contexts:
        context_text = ' '.join(contexts[:5]).lower()  # First 5 contexts
        context_words = set(context_text.split())
        answer_words = set(answer_lower.split())
        overlap = len(context_words.intersection(answer_words))
        metrics['context_word_overlap'] = overlap
        metrics['context_usage_ratio'] = overlap / max(len(answer_words), 1)
        
        # Check for direct quotes or paraphrases from context
        context_sentences = re.split(r'[.!?]+', context_text)
        answer_sentences = re.split(r'[.!?]+', answer_lower)
        similar_sentences = 0
        for ans_sent in answer_sentences:
            if len(ans_sent.split()) > 5:  # Only check substantial sentences
                for ctx_sent in context_sentences:
                    if len(ctx_sent.split()) > 5:
                        # Simple similarity check
                        ans_words = set(ans_sent.split())
                        ctx_words = set(ctx_sent.split())
                        if len(ans_words.intersection(ctx_words)) / max(len(ans_words), 1) > 0.3:
                            similar_sentences += 1
                            break
        metrics['context_based_sentences'] = similar_sentences
        metrics['context_reliance_ratio'] = similar_sentences / max(len(answer_sentences), 1)
    else:
        metrics['context_word_overlap'] = 0
        metrics['context_usage_ratio'] = 0
        metrics['context_based_sentences'] = 0
        metrics['context_reliance_ratio'] = 0
    
    # Ground truth similarity
    gt_words = set(ground_truth.lower().split())
    answer_words = set(answer_lower.split())
    answer_gt_overlap = len(gt_words.intersection(answer_words))
    metrics['ground_truth_word_overlap'] = answer_gt_overlap
    metrics['ground_truth_similarity'] = answer_gt_overlap / max(len(gt_words), 1)
    
    # Answer structure and quality
    metrics['has_introduction'] = 1 if answer_lower.startswith(('based on', 'according to', 'the document', 'the analysis')) else 0
    metrics['has_conclusion'] = 1 if any(word in answer_lower[-100:] for word in ['conclusion', 'summary', 'overall', 'in summary']) else 0
    metrics['has_structure'] = 1 if any(word in answer_lower for word in ['first', 'second', 'third', 'additionally', 'furthermore', 'however']) else 0
    
    # Completeness (check if answer addresses key question components)
    question_lower = question.lower()
    question_keywords = set(question_lower.split())
    question_answer_overlap = len(question_keywords.intersection(answer_words))
    metrics['question_coverage'] = question_answer_overlap / max(len(question_keywords), 1)
    
    # Specificity score (ratio of specific terms to general terms)
    specific_terms = ['million', 'tonnes', '2030', 'kg', 'percent', '%', 'dollar', '$', 
                     'policy', 'framework', 'mission', 'incentive', 'target']
    general_terms = ['the', 'is', 'are', 'and', 'or', 'but', 'a', 'an', 'in', 'on', 'at', 'to', 'for']
    specific_count = sum(1 for term in specific_terms if term in answer_lower)
    general_count = sum(1 for term in general_terms if term in answer_lower)
    metrics['specificity_ratio'] = specific_count / max(general_count, 1) if general_count > 0 else specific_count
    
    # Readability (simple Flesch-like metric based on sentence and word length)
    avg_sentence_length = metrics['avg_words_per_sentence']
    metrics['readability_score'] = max(0, min(100, 100 - (avg_sentence_length - 10) * 2))
    
    # RAGAS-STYLE METRICS (Manual Implementation)
    # 1. Context Precision: Proportion of retrieved contexts that are relevant
    if contexts:
        relevant_contexts = 0
        question_gt_keywords = set(question_lower.split()) | set(ground_truth.lower().split())
        
        for ctx in contexts[:10]:  # Check first 10 contexts
            ctx_lower = ctx.lower()
            ctx_words = set(ctx_lower.split())
            overlap_ratio = len(question_gt_keywords.intersection(ctx_words)) / max(len(question_gt_keywords), 1)
            if overlap_ratio > 0.1:  # At least 10% keyword overlap
                relevant_contexts += 1
        
        metrics['context_precision'] = relevant_contexts / max(len(contexts), 1)
    else:
        metrics['context_precision'] = 0.0
    
    # 2. Context Recall: Proportion of relevant information from ground truth found in contexts
    if contexts:
        all_context_text = ' '.join(contexts).lower()
        context_words = set(all_context_text.split())
        gt_words_set = set(ground_truth.lower().split())
        
        gt_words_in_context = len(gt_words_set.intersection(context_words))
        metrics['context_recall'] = gt_words_in_context / max(len(gt_words_set), 1)
    else:
        metrics['context_recall'] = 0.0
    
    # 3. Faithfulness: Measures if answer is grounded in context (no hallucinations)
    if contexts:
        all_context_text = ' '.join(contexts).lower()
        answer_sentences = [s.strip() for s in re.split(r'[.!?]+', answer_lower) if len(s.strip()) > 10]
        context_sentences = [s.strip() for s in re.split(r'[.!?]+', all_context_text) if len(s.strip()) > 10]
        
        faithful_sentences = 0
        for ans_sent in answer_sentences:
            ans_words = set(ans_sent.split())
            for ctx_sent in context_sentences:
                ctx_words = set(ctx_sent.split())
                overlap = len(ans_words.intersection(ctx_words))
                if overlap / max(len(ans_words), 1) > 0.3:
                    faithful_sentences += 1
                    break
        
        metrics['faithfulness'] = faithful_sentences / max(len(answer_sentences), 1) if answer_sentences else 0.0
    else:
        metrics['faithfulness'] = 0.0
    
    # 4. Answer Correctness: How well the answer matches the ground truth
    gt_words_set = set(ground_truth.lower().split())
    answer_words_set = set(answer_lower.split())
    
    word_overlap = len(gt_words_set.intersection(answer_words_set))
    word_precision = word_overlap / max(len(answer_words_set), 1)
    word_recall = word_overlap / max(len(gt_words_set), 1)
    
    if word_precision + word_recall > 0:
        metrics['answer_correctness'] = 2 * (word_precision * word_recall) / (word_precision + word_recall)
    else:
        metrics['answer_correctness'] = 0.0
    
    metrics['answer_precision'] = word_precision
    metrics['answer_recall'] = word_recall
    
    # Check for key facts from ground truth
    gt_key_facts = [
        'green hydrogen', 'cost', 'benchmark', 'India', 'policy',
        'electrolyzer', 'renewable', '2030', 'million', 'tonnes'
    ]
    facts_in_answer = sum(1 for fact in gt_key_facts if fact in answer_lower)
    metrics['key_facts_coverage'] = facts_in_answer / len(gt_key_facts)
    
    return metrics

# Ensure variables are defined
if 'complex_query_adv' not in globals():
    complex_query_adv = "What are the key cost benchmarks and performance metrics for green hydrogen production in India?"

if 'ground_truth_answer_adv' not in globals():
    ground_truth_answer_adv = """Based on the green hydrogen benchmarking document, key cost benchmarks for green hydrogen production in India include electrolyzer costs, renewable energy prices, and infrastructure requirements. The document identifies that green hydrogen costs range from $3-7 per kg depending on scale and renewable energy source. Policy frameworks such as India's National Green Hydrogen Mission with targets of 5 million tonnes by 2030, along with production-linked incentives, are key factors supporting adoption. Main challenges include high capital costs and infrastructure needs, while opportunities include abundant renewable resources and government support."""

baseline_answer = baseline_result if 'baseline_result' in globals() else "No baseline answer available"
advanced_answer = ""
if 'final_state' in locals() and final_state:
    advanced_answer = final_state.get('final_answer', '')
elif 'baseline_result' in globals():
    advanced_answer = baseline_result

if not baseline_answer or baseline_answer == "No baseline answer available":
    baseline_answer = "Answer not available"

if not advanced_answer:
    advanced_answer = baseline_answer

# Get contexts
baseline_contexts_list = baseline_contexts[0] if 'baseline_contexts' in locals() and baseline_contexts else []
advanced_contexts_list = advanced_contexts[0] if 'advanced_contexts' in locals() and advanced_contexts else []

# Calculate metrics for both models
print("\nCalculating comprehensive metrics...")
baseline_metrics = comprehensive_evaluation(
    complex_query_adv, 
    baseline_answer, 
    ground_truth_answer_adv, 
    baseline_contexts_list,
    "Baseline RAG"
)

advanced_metrics = comprehensive_evaluation(
    complex_query_adv, 
    advanced_answer, 
    ground_truth_answer_adv, 
    advanced_contexts_list,
    "Deep Thinking RAG"
)

# ============================================================================
# DISPLAY RESPONSES ONE AFTER THE OTHER
# ============================================================================
print("\n" + "=" * 80)
print("QUERY")
print("=" * 80)
print(f"{complex_query_adv}")

print("\n" + "=" * 80)
print("BASELINE RAG RESPONSE")
print("=" * 80)
print(f"Answer Length: {len(baseline_answer)} characters, {len(baseline_answer.split())} words")
print(f"Retrieved Contexts: {len(baseline_contexts_list)} chunks")
print(f"Sentences: {baseline_metrics['sentence_count']}")
print("\n" + "-" * 80)
print("RESPONSE:")
print("-" * 80)
print(baseline_answer)
print("-" * 80)

print("\n" + "=" * 80)
print("DEEP THINKING RAG RESPONSE")
print("=" * 80)
print(f"Answer Length: {len(advanced_answer)} characters, {len(advanced_answer.split())} words")
print(f"Retrieved Contexts: {len(advanced_contexts_list)} chunks")
print(f"Sentences: {advanced_metrics['sentence_count']}")
print("\n" + "-" * 80)
print("RESPONSE:")
print("-" * 80)
print(advanced_answer)
print("-" * 80)

# ============================================================================
# ALL METRICS COMPARISON TABLE
# ============================================================================
print("\n" + "=" * 80)
print("ALL METRICS COMPARISON")
print("=" * 80)

# Create comprehensive comparison table with all metrics
all_metrics = [
    # Basic Metrics
    ('answer_length', 'Answer Length (characters)'),
    ('word_count', 'Word Count'),
    ('sentence_count', 'Sentence Count'),
    ('avg_words_per_sentence', 'Average Words per Sentence'),
    
    # Content Quality
    ('key_terms_found', 'Key Terms Found'),
    ('key_terms_coverage', 'Key Terms Coverage'),
    ('numerical_data_points', 'Numerical Data Points'),
    ('has_specific_values', 'Has Specific Values (0/1)'),
    ('key_facts_coverage', 'Key Facts Coverage'),
    
    # RAGAS-Style Metrics
    ('context_precision', 'Context Precision (RAGAS)'),
    ('context_recall', 'Context Recall (RAGAS)'),
    ('faithfulness', 'Faithfulness (RAGAS)'),
    ('answer_correctness', 'Answer Correctness (RAGAS)'),
    ('answer_precision', 'Answer Precision'),
    ('answer_recall', 'Answer Recall'),
    
    # Context Usage
    ('context_word_overlap', 'Context Word Overlap'),
    ('context_usage_ratio', 'Context Usage Ratio'),
    ('context_based_sentences', 'Context-Based Sentences'),
    ('context_reliance_ratio', 'Context Reliance Ratio'),
    
    # Answer Quality
    ('ground_truth_similarity', 'Ground Truth Similarity'),
    ('question_coverage', 'Question Coverage'),
    ('specificity_ratio', 'Specificity Ratio'),
    ('readability_score', 'Readability Score'),
    
    # Structure
    ('has_introduction', 'Has Introduction (0/1)'),
    ('has_conclusion', 'Has Conclusion (0/1)'),
    ('has_structure', 'Has Structure (0/1)'),
]

comparison_data = {
    'Metric': [],
    'Baseline RAG': [],
    'Deep Thinking RAG': [],
    'Improvement': []
}

for metric_key, metric_name in all_metrics:
    baseline_val = baseline_metrics.get(metric_key, 0)
    advanced_val = advanced_metrics.get(metric_key, 0)
    
    # Calculate improvement
    if isinstance(baseline_val, (int, float)) and baseline_val > 0:
        improvement = ((advanced_val - baseline_val) / baseline_val) * 100
        improvement_str = f"{improvement:+.1f}%"
    elif isinstance(baseline_val, (int, float)) and baseline_val == 0 and advanced_val > 0:
        improvement_str = "∞ (from 0)"
    else:
        improvement_str = "N/A"
    
    # Format values
    if isinstance(baseline_val, float) and 0 < baseline_val < 1:
        baseline_str = f"{baseline_val:.3f}"
        advanced_str = f"{advanced_val:.3f}"
    elif isinstance(baseline_val, float):
        baseline_str = f"{baseline_val:.1f}"
        advanced_str = f"{advanced_val:.1f}"
    else:
        baseline_str = str(baseline_val)
        advanced_str = str(advanced_val)
    
    comparison_data['Metric'].append(metric_name)
    comparison_data['Baseline RAG'].append(baseline_str)
    comparison_data['Deep Thinking RAG'].append(advanced_str)
    comparison_data['Improvement'].append(improvement_str)

comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================
print("\n" + "=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)

# Calculate overall scores
baseline_overall = (
    baseline_metrics['key_terms_coverage'] * 0.2 +
    baseline_metrics['ground_truth_similarity'] * 0.2 +
    baseline_metrics['context_usage_ratio'] * 0.15 +
    baseline_metrics['question_coverage'] * 0.15 +
    baseline_metrics['context_precision'] * 0.1 +
    baseline_metrics['context_recall'] * 0.1 +
    baseline_metrics['faithfulness'] * 0.05 +
    baseline_metrics['answer_correctness'] * 0.05
) * 100

advanced_overall = (
    advanced_metrics['key_terms_coverage'] * 0.2 +
    advanced_metrics['ground_truth_similarity'] * 0.2 +
    advanced_metrics['context_usage_ratio'] * 0.15 +
    advanced_metrics['question_coverage'] * 0.15 +
    advanced_metrics['context_precision'] * 0.1 +
    advanced_metrics['context_recall'] * 0.1 +
    advanced_metrics['faithfulness'] * 0.05 +
    advanced_metrics['answer_correctness'] * 0.05
) * 100

summary_data = {
    'Metric': [
        'Overall Score (weighted)',
        'Key Terms Coverage',
        'Ground Truth Similarity',
        'Context Usage',
        'Question Coverage',
        'Context Precision (RAGAS)',
        'Context Recall (RAGAS)',
        'Faithfulness (RAGAS)',
        'Answer Correctness (RAGAS)',
        'Specificity',
        'Readability'
    ],
    'Baseline RAG': [
        f"{baseline_overall:.1f}%",
        f"{baseline_metrics['key_terms_coverage']:.1%}",
        f"{baseline_metrics['ground_truth_similarity']:.1%}",
        f"{baseline_metrics['context_usage_ratio']:.1%}",
        f"{baseline_metrics['question_coverage']:.1%}",
        f"{baseline_metrics['context_precision']:.3f}",
        f"{baseline_metrics['context_recall']:.3f}",
        f"{baseline_metrics['faithfulness']:.3f}",
        f"{baseline_metrics['answer_correctness']:.3f}",
        f"{baseline_metrics['specificity_ratio']:.2f}",
        f"{baseline_metrics['readability_score']:.1f}"
    ],
    'Deep Thinking RAG': [
        f"{advanced_overall:.1f}%",
        f"{advanced_metrics['key_terms_coverage']:.1%}",
        f"{advanced_metrics['ground_truth_similarity']:.1%}",
        f"{advanced_metrics['context_usage_ratio']:.1%}",
        f"{advanced_metrics['question_coverage']:.1%}",
        f"{advanced_metrics['context_precision']:.3f}",
        f"{advanced_metrics['context_recall']:.3f}",
        f"{advanced_metrics['faithfulness']:.3f}",
        f"{advanced_metrics['answer_correctness']:.3f}",
        f"{advanced_metrics['specificity_ratio']:.2f}",
        f"{advanced_metrics['readability_score']:.1f}"
    ],
    'Improvement': [
        f"{((advanced_overall - baseline_overall) / baseline_overall * 100):+.1f}%" if baseline_overall > 0 else "N/A",
        f"{((advanced_metrics['key_terms_coverage'] - baseline_metrics['key_terms_coverage']) / baseline_metrics['key_terms_coverage'] * 100):+.1f}%" if baseline_metrics['key_terms_coverage'] > 0 else "N/A",
        f"{((advanced_metrics['ground_truth_similarity'] - baseline_metrics['ground_truth_similarity']) / baseline_metrics['ground_truth_similarity'] * 100):+.1f}%" if baseline_metrics['ground_truth_similarity'] > 0 else "N/A",
        f"{((advanced_metrics['context_usage_ratio'] - baseline_metrics['context_usage_ratio']) / baseline_metrics['context_usage_ratio'] * 100):+.1f}%" if baseline_metrics['context_usage_ratio'] > 0 else "N/A",
        f"{((advanced_metrics['question_coverage'] - baseline_metrics['question_coverage']) / baseline_metrics['question_coverage'] * 100):+.1f}%" if baseline_metrics['question_coverage'] > 0 else "N/A",
        f"{((advanced_metrics['context_precision'] - baseline_metrics['context_precision']) / baseline_metrics['context_precision'] * 100):+.1f}%" if baseline_metrics['context_precision'] > 0 else "N/A",
        f"{((advanced_metrics['context_recall'] - baseline_metrics['context_recall']) / baseline_metrics['context_recall'] * 100):+.1f}%" if baseline_metrics['context_recall'] > 0 else "N/A",
        f"{((advanced_metrics['faithfulness'] - baseline_metrics['faithfulness']) / baseline_metrics['faithfulness'] * 100):+.1f}%" if baseline_metrics['faithfulness'] > 0 else "N/A",
        f"{((advanced_metrics['answer_correctness'] - baseline_metrics['answer_correctness']) / baseline_metrics['answer_correctness'] * 100):+.1f}%" if baseline_metrics['answer_correctness'] > 0 else "N/A",
        f"{((advanced_metrics['specificity_ratio'] - baseline_metrics['specificity_ratio']) / baseline_metrics['specificity_ratio'] * 100):+.1f}%" if baseline_metrics['specificity_ratio'] > 0 else "N/A",
        f"{((advanced_metrics['readability_score'] - baseline_metrics['readability_score']) / baseline_metrics['readability_score'] * 100):+.1f}%" if baseline_metrics['readability_score'] > 0 else "N/A"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + summary_df.to_string(index=False))

# ============================================================================
# KEY FINDINGS
# ============================================================================
print("\n" + "=" * 80)
print("KEY FINDINGS")
print("=" * 80)

findings = []
if advanced_overall > baseline_overall:
    findings.append(f"✓ Deep Thinking RAG shows {((advanced_overall - baseline_overall) / baseline_overall * 100):.1f}% improvement in overall score")
else:
    findings.append(f"⚠ Deep Thinking RAG shows {((advanced_overall - baseline_overall) / baseline_overall * 100):.1f}% change in overall score")

if advanced_metrics['key_terms_found'] > baseline_metrics['key_terms_found']:
    findings.append(f"✓ Deep Thinking RAG covers {advanced_metrics['key_terms_found'] - baseline_metrics['key_terms_found']} more key terms")

if advanced_metrics['context_precision'] > baseline_metrics['context_precision']:
    findings.append(f"✓ Context Precision improved: {baseline_metrics['context_precision']:.3f} → {advanced_metrics['context_precision']:.3f}")

if advanced_metrics['context_recall'] > baseline_metrics['context_recall']:
    findings.append(f"✓ Context Recall improved: {baseline_metrics['context_recall']:.3f} → {advanced_metrics['context_recall']:.3f}")

if advanced_metrics['faithfulness'] > baseline_metrics['faithfulness']:
    findings.append(f"✓ Faithfulness improved: {baseline_metrics['faithfulness']:.3f} → {advanced_metrics['faithfulness']:.3f} (less hallucination)")

if advanced_metrics['answer_correctness'] > baseline_metrics['answer_correctness']:
    findings.append(f"✓ Answer Correctness improved: {baseline_metrics['answer_correctness']:.3f} → {advanced_metrics['answer_correctness']:.3f}")

if advanced_metrics['numerical_data_points'] > baseline_metrics['numerical_data_points']:
    findings.append(f"✓ Deep Thinking RAG includes {advanced_metrics['numerical_data_points'] - baseline_metrics['numerical_data_points']} more numerical data points")

if advanced_metrics['context_usage_ratio'] > baseline_metrics['context_usage_ratio']:
    findings.append(f"✓ Deep Thinking RAG uses context more effectively ({advanced_metrics['context_usage_ratio']:.1%} vs {baseline_metrics['context_usage_ratio']:.1%})")

for finding in findings:
    print(f"  {finding}")

print("\n" + "=" * 80)

COMPREHENSIVE EVALUATION - Baseline vs Deep Thinking RAG

Calculating comprehensive metrics...

QUERY
What are the key cost benchmarks and performance metrics for green hydrogen production in India as outlined in the benchmarking document? Compare these to conventional hydrogen production methods and identify the main factors affecting cost competitiveness.

BASELINE RAG RESPONSE
Answer Length: 861 characters, 126 words
Retrieved Contexts: 0 chunks
Sentences: 5

--------------------------------------------------------------------------------
RESPONSE:
--------------------------------------------------------------------------------
The provided context does not contain specific information regarding the cost benchmarks and performance metrics for green hydrogen production in India, nor does it compare these to conventional hydrogen production methods. The text primarily discusses the challenges and potential of renewable energy sources, particularly solar energy, in India, along with th

### Restart Runtime

**IMPORTANT:** Please restart your Colab runtime now (`Runtime > Restart runtime...`).

After the runtime has restarted, you **must** re-run **all** setup cells from the very beginning of the notebook (`part1-2-code-pro` onwards) up to `part6-4-code-impl-pro-adv` to ensure all components are re-initialized with the correct `ragas` environment.

In [108]:
!pip show ragas

zsh:1: command not found: pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### 6.3. Interpreting the Evaluation Scores for Our Advanced Pipeline

The quantitative results provide a definitive verdict on the superiority of the Deep Thinking architecture:

-   **Context Precision (0.50 vs 1.00):** The baseline's context was only partially relevant, as it could only retrieve general information about competition without the crucial details on AMD's 2024 strategy. The advanced agent's multi-step, multi-tool retrieval achieved a perfect score.
-   **Context Recall (0.33 vs 1.00):** The baseline retriever completely missed the information from the web, resulting in a very low recall score. The advanced agent's planning and tool-use ensured all necessary information from all sources was queried, achieving perfect recall.
-   **Faithfulness (1.00 vs 1.00):** Both systems were highly faithful to the context they were given. The baseline correctly stated it didn't have the information, and the advanced agent correctly used the information it found.
-   **Answer Correctness (0.40 vs 0.99):** This is the ultimate measure of quality. The baseline's answer was less than 40% correct because it was missing the entire second half of the required analysis. The advanced agent's answer was nearly perfect, demonstrating its ability to perform true synthesis across multiple knowledge sources.

**Conclusion:** The evaluation provides objective, quantitative proof that the architectural shift to a cyclical, tool-aware, and adaptive reasoning agent results in a dramatic and measurable improvement in performance on complex, real-world queries.

## Part 7: Optimizations and Production Considerations

### 7.1. Optimization 1: Implementing a Cache for Repeated Sub-Queries

Our agent makes multiple calls to expensive LLMs (Planner, Rewriter, etc.). In a production environment where users may ask similar questions, caching these calls is essential for performance and cost management. LangChain provides built-in caching that can be easily integrated with our agents.

```python
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# To enable caching for all LLM calls in the session
set_llm_cache(InMemoryCache())
```

### 7.2. Feature 1: Provenance and Citations - Building User Trust

Users need to trust the answers our agent provides. A critical feature for production is **provenance**. We have implemented this in our `final_answer_node`. By explicitly prompting the final LLM to use the source metadata (`section` title or `URL`) attached to each piece of evidence, we generate citations directly in the final answer. This makes the agent's reasoning transparent and verifiable across all its knowledge sources.

### 7.3. Discussion: The Next Level - MDPs and Learned Policies (The DeepRAG Paper)

Currently, our Policy and Supervisor Agents use a powerful, general-purpose LLM to make decisions. While highly effective, this can be slow and costly. The academic frontier, as explored in papers like DeepRAG, frames this reasoning process as a **Markov Decision Process (MDP)**. By logging thousands of successful and unsuccessful reasoning traces from our LangSmith project, we could use reinforcement learning to train smaller, specialized 'policy models'. A learned policy could make the `CONTINUE`/`FINISH` decision or the `vector`/`keyword` decision much faster and more cheaply than a full GPT-4o call, while being highly optimized for our specific domain.

### 7.4. Handling Failure: Graceful Exits and Fallbacks When No Answer is Found

A production system must be robust to failure. What if a sub-question yields no relevant documents? Our current agent simply logs this and moves on. A more advanced implementation would involve:
1.  **Reflection with Failure Recognition:** The reflection agent could be prompted to recognize when context is insufficient and explicitly state that the sub-question could not be answered.
2.  **`REVISE_PLAN` Path:** The policy agent could have a third option, `REVISE_PLAN`. This would route the state back to the `plan_node`, but this time with the full history, prompting it to create a new, better plan to overcome the dead end.
3.  **Graceful Exit:** If re-planning also fails, the graph should route to a final `no_answer_node` that explicitly informs the user that a confident answer could not be constructed from the available documents.

## Part 8: Conclusion and Key Takeaways

### 8.1. Summary of Our Journey

In this notebook, we have undertaken a complete journey from a rudimentary RAG pipeline to a sophisticated autonomous reasoning agent. We began by demonstrating the inherent limitations of a shallow, single-pass architecture on a complex, multi-source query. We then systematically constructed a **Deep Thinking RAG** system, adding layers of intelligence: a tool-aware strategic planner, an adaptive, high-fidelity multi-stage retrieval funnel, external tool augmentation, and a self-critiquing policy engine. By orchestrating this advanced cognitive architecture with LangGraph, we created a system capable of true, multi-source synthesis. Our final, rigorous evaluation with RAGAs provided objective, quantitative proof of its dramatic superiority over the baseline.

### 8.2. Key Architectural Principles of Advanced RAG Systems

1.  **Stateful Cyclical Reasoning:** The fundamental shift is from linear, stateless chains to cyclical, stateful graphs. Intelligence emerges from the ability to iterate, reflect, and refine.
2.  **Decomposition is King:** Complex problems must be broken down. An explicit, structured planning step is the most critical element for tackling multi-hop, multi-source queries.
3.  **Tool Augmentation for Comprehensive Knowledge:** No single knowledge source is sufficient. Agents must be able to reason about when their internal knowledge is lacking and autonomously select external tools (like web search) to fill the gaps.
4.  **Dynamic Strategy Selection:** Rigidity is fragile. Empowering the agent to dynamically adapt its strategies (e.g., choosing a retrieval method) based on the specific task at hand leads to more efficient and accurate results.
5.  **Separation of Recall and Precision:** Retrieval is not a single step. A multi-stage funnel that first maximizes recall and then maximizes precision (Reranking) is essential for finding the right evidence.
6.  **Explicit Self-Correction:** A dedicated policy or 'judge' component that inspects progress and controls the loop is the key to autonomy and robustness.

### 8.3. Future Directions and Further Reading

This architecture serves as a powerful and extensible template. Future work could include:
-   **Multi-Document Analysis:** Extending the agent to answer questions that require synthesizing information across a *corpus* of documents, not just a single one, by adding a preliminary 'document routing' step.
-   **Structured Tool Use:** Empowering the agent with tools to query structured databases (e.g., SQL) or financial data APIs, and allowing the planner to generate the necessary code or queries for those tools.
-   **Fine-Tuning a Supervisor Model:** Training a smaller, specialized SLM on traces from LangSmith to perform the Retrieval Supervisor's role, leading to significant cost and latency reductions in production.

# Task
Execute cell `part4-2-code-pro-adv`, then cell `part4-3-build-pro-adv`, and finally cell `part4-4-viz-pro-adv`.

## run_part4-2-code-pro-adv

### Subtask:
Execute cell `part4-2-code-pro-adv` to define the conditional edge logic functions.


**Reasoning**:
The user explicitly asked to execute the cell `part4-2-code-pro-adv` to define the conditional edge logic functions. I will provide the content of this cell in a code block.



**Reasoning**:
Now that the conditional edge logic functions are defined, the next step is to execute cell `part4-3-build-pro-adv` to build the `StateGraph` as per the main task.



**Reasoning**:
The StateGraph has been successfully constructed. The next step, as per the main task, is to compile the graph and attempt to visualize it by executing cell `part4-4-viz-pro-adv`.



In [34]:
deep_thinking_rag_graph = graph.compile()
print("Graph compiled successfully.")

try:
    from IPython.display import Image, display
    # Correctly call get_graph() before draw_png()
    png_image = deep_thinking_rag_graph.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}. Please ensure pygraphviz is installed.")

Graph compiled successfully.
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed.


## Summary:

### Data Analysis Key Findings

*   **Conditional Edge Logic Functions Defined**: Two critical conditional edge logic functions, `route_by_tool` and `should_continue_node`, were successfully defined for dynamic graph routing and iteration control.
*   **StateGraph Constructed**: The RAG process's `StateGraph` was successfully built, incorporating nodes for planning, retrieval (10k and web), reranking, compression, reflection, and final answer generation, along with a `choose_next_tool` routing point. Edges, both direct and conditional, were established to manage the flow between these nodes.
*   **Graph Compilation Successful**: The constructed `StateGraph` was successfully compiled into `deep_thinking_rag_graph`, making it ready for execution.
*   **Graph Visualization Failed**: While the graph compiled successfully, its visualization failed due to the absence of the `pygraphviz` library, which is required for rendering the graph diagram.

### Insights or Next Steps

*   To fully understand and debug the flow of the RAG graph, install `pygraphviz` to enable graph visualization.
*   The RAG graph is now fully constructed and compiled, ready for execution with a question input.


# Task
Modify cell `part6-4-code-impl-pro-adv` to re-add the import for `RunConfig` and pass `RunConfig()` to the `RagasCompatibleHuggingFaceBgeEmbeddings` constructor to resolve the `ValidationError`.

## modify_part6-4-code-impl-pro-adv

### Subtask:
Re-add the import for RunConfig and pass it to the RagasCompatibleHuggingFaceBgeEmbeddings constructor.


## Summary:

### Data Analysis Key Findings
*   A `ValidationError` was encountered when initializing `RagasCompatibleHuggingFaceBgeEmbeddings`.
*   The resolution involved re-importing `RunConfig` and passing `RunConfig()` as an argument to the `RagasCompatibleHuggingFaceBgeEmbeddings` constructor.

### Insights or Next Steps
*   Ensure all necessary imports are present and correctly passed as arguments, especially when dealing with complex object initializations to avoid `ValidationError`s.
